### Trừ tạm		
        TH1: Đã xuất GCN nhỏ hơn số cần TC -> ( Tồn kho + Đã xuất gcn + đã đặt (chưa về)) - ( Cần + cần of TC ) => Trừ tạm
        TH2: (Đã xuất GCN lớn hơn Số cần TC) -> Số cần TC = 0 (tự triệt tiêu) =>  Số tồn - Số cần => Trừ tạm


##### Cần mua:
- TH1 : Cần + cần of TC - Xuất GCN – Kho > Đã đặt => Cần + cần of TC - Xuất GCN – Kho					
- TH2 : Cần + cần of TC - Xuất GCN – Kho < Đã đặt => Đã đặt					


	- Đã đặt: 
		- purchase order 
		- purchase_order_detail: purchase order id
		- purchase_order_detail_status
		- packing_list: product_id for purchase_order_id: Note: Xuất bù, bổ sung, ... đã cập nhập thẳng
		- product_detail
		- Đã đặt là đặt nhưng chưa nhập kho, trừ cho lượng nhập kho ra lượng cần đặt

	- Tồn kho:


	- Đã xuất GCN: Note: Xuất cấu thành -> Nhập về thứ cấp -> Nhập vào kho.




	- Công thức tính số cần:
		- Cần - XKTT:	
			- Đơn hàng bán(múi + SA): sau khi nghiep vu tao dinh muc 
			- PYC Xuất kho
			- YC xuất bù LSX
			- Xuất đổi trả(chỉ tính sơ cấp)
			
		- Cần của Thứ cấp 
			- Đơn hàng bán của TC
			- Phiếu yêu cầu xuất kho của TC
			- YC xuất bù LSX

In [4]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
dfReqStockoutProductDetail = pd.read_csv("../data_csv/ReqStockoutProductDetail.csv") # Request Stockout Product Detail
dfOutSourcingDetail = pd.read_csv("../data_csv/OutSourcingDetail.csv") # Outsourcing Detail
dfReturnProduct = pd.read_csv("../data_csv/ReturnProduct.csv") # Return Product
dfPurchaseOrder = pd.read_csv("../data_csv/PurchaseOrder.csv") # Purchase Order
dfStockInPurchaseOrder = pd.read_csv("../data_csv/StockInPurchaseOrder.csv") # Purchase Order
dfOutSourcingQuota = pd.read_csv("../data_csv/OutSourcingQuota.csv")
# dfOutSourcing = pd.read_csv("OutSourcing.csv")
dfReqStockInProduct = pd.read_csv("../data_csv/ReqStockInProductStatus.csv")
dfProductStockInRealDetails = pd.read_csv('../data_csv/ProductStockInRealDetails.csv')
dfProductionStockOut = pd.read_csv('../data_csv/ProductionStockOut.csv')
dfWhCorrectionDetail = pd.read_csv('../data_csv/WhCorrectionDetail.csv')

dfRelationship = pd.read_csv('../data_csv/Relationship.csv')
dfMaterialReports = pd.read_csv('../data_csv/MaterialReports.csv')


### <input type="checkbox" disabled checked > Nguyên phụ liệu sơ cấp + thứ cấp

In [6]:
dfProduct = pd.read_csv("../data_csv/Product.csv")

In [7]:
dfPrimary = dfProduct.loc[dfProduct['is_outsourcing'] == 0]
dfSecondary = dfProduct.loc[dfProduct['is_outsourcing'] == 1]

In [8]:
dfPrimary[dfPrimary['product_id'] == 793]

,product_id,is_outsourcing,product_code,quantity_product
756,793,0,1-KATE00-BLK-00-HV,2028.4476


### <input type="checkbox" disabled checked > Tỉ lệ chuyển đổi nguyên phụ liệu + Bug 1 NPL có nhiều tỉ lệ chuyển đổi 

In [9]:
dfProductUnit = pd.read_csv('../data_csv/ProductUnit.csv')
dfProductUnit.reset_index(drop = True)

,product_id,product_unit_id,unit_id,is_default
0,21,21,1,1
1,22,22,1,1
2,26,26,1,1
3,28,28,1,1
4,31,31,1,1
...,...,...,...,...
16842,11468,21746,1,1
16843,11469,21747,1,1
16844,11470,21748,4,1
16845,11471,21749,4,1


In [10]:
dfProductUnitExchange = pd.read_csv('../data_csv/ProductUnitExchange.csv')
# dfProductUnitExchange = dfProductUnitExchange.drop_duplicates(subset=['product_id' , 'source_unit_id' , 'result_unit_id'], keep='first').reset_index(drop= True)
dfProductUnitExchange

,product_id,source_unit_id,result_unit_id,source_quantity,result_quantity
0,6773,4,14,110.0000,1.0
1,6758,4,14,110.0000,1.0
2,6773,4,14,115.0000,1.0
3,2348,1,5,0.9144,1.0
4,194,1,5,0.9144,1.0
...,...,...,...,...,...
6963,11450,1,5,0.9144,1.0
6964,11449,1,5,0.9144,1.0
6965,11445,1,5,0.9144,1.0
6966,11455,1,5,0.9140,1.0


In [11]:
dfProductUnitExchange = dfProductUnitExchange.drop_duplicates(subset=['product_id' , 'source_unit_id' , 'result_unit_id'], keep='first').reset_index(drop= True)

In [691]:
df2 = dfProductUnitExchange[dfProductUnitExchange[['product_id' , 'source_quantity']].duplicated() == True]
df2

,product_id,source_unit_id,result_unit_id,source_quantity,result_quantity


## <input type="checkbox" disabled checked > Đã đặt `orderedQuantity`

#### <input type="checkbox" disabled checked > ĐH thu mua + bug tạo đơn hàng trùng NPL, khác tỉ lệ chuyển đổi, tổng đã nhập `dfPurchaseOrderDetailSum`

In [692]:
dfPurchaseOrderDetail = pd.read_csv("../data_csv/PurchaseOrderDetail.csv") # Purchase Order
dfStockInPurchaseOrder = pd.read_csv("../data_csv/StockInPurchaseOrder.csv") # Purchase Order
dfPurchaseOrder = pd.read_csv('../data_csv/PurchaseOrder.csv')
dfPurchaseOrder

,total_amount,purchase_order_id,is_completed,po_status
0,617.0,1,1,completed
1,1000.0,2,1,completed
2,2500.0,3,1,completed
3,100.0,4,1,completed
4,2000.0,5,1,completed
...,...,...,...,...
1332,15100.0,1372,0,approval
1333,1000.0,1373,0,review
1334,7.0,1374,1,completed
1335,9100.0,1375,0,approval


In [693]:
df = dfPurchaseOrderDetail[dfPurchaseOrderDetail[['product_id' ,'purchase_order_id']].duplicated() == True]

In [694]:
df = dfStockInPurchaseOrder[~dfStockInPurchaseOrder['purchase_order_id'].isin(dfPurchaseOrderDetail['purchase_order_id'].to_list())].reset_index(drop = True)

In [695]:
dfPurchaseOrderDetail.at[426, 'is_complete'] = 1
dfPurchaseOrderDetail.at[426, 'is_complete'] = 1
dfPurchaseOrderDetail.at[1785, 'is_complete'] = 1
dfPurchaseOrderDetail.at[1787, 'is_complete'] = 1
dfPurchaseOrderDetail.at[19, 'is_complete'] = 1

In [696]:
dfPurchaseOrder[dfPurchaseOrder['purchase_order_id'] == 1703]

,total_amount,purchase_order_id,is_completed,po_status


In [697]:
dfPurchaseOrderTotal = dfPurchaseOrder.merge(dfPurchaseOrderDetail, on = 'purchase_order_id', how = 'left').replace(np.NaN , 0)
dfPurchaseOrderTotal[dfPurchaseOrderTotal['product_id'] == 1703]

,total_amount,purchase_order_id,is_completed,po_status,product_id,product_unit_id,purchase_order_quantity,is_complete
2171,9600.0,575,0,approval,1703.0,13558.0,200.0,0.0
4577,8600.0,1180,0,approval,1703.0,13558.0,50.0,0.0


In [698]:
dfPurchaseOrderTotal = dfPurchaseOrder.merge(dfPurchaseOrderDetail, on = 'purchase_order_id', how = 'left').replace(np.NaN , 0)
dfPurchaseOrderTotalNotComplete = dfPurchaseOrderTotal[(dfPurchaseOrderTotal['is_completed'] != 1) & (dfPurchaseOrderTotal['is_complete'] != 1)
                                                       & (dfPurchaseOrderTotal['po_status'] == 'approval')]
dfPurchaseOrderDetail = dfPurchaseOrderTotalNotComplete.merge(dfProductUnit, on = ['product_id' , 'product_unit_id'], how = 'left')
dfPurchaseOrderDetail['quantity_exchange'] = 0
# dfPurchaseOrderDetail[dfPurchaseOrderDetail['product_id'] == 6823]

In [699]:
dfProductUnitExchange[dfProductUnitExchange['product_id'] == 6823]

,product_id,source_unit_id,result_unit_id,source_quantity,result_quantity
3773,6823,1,5,0.9144,1.0


In [700]:
for i in range(0 , dfPurchaseOrderDetail.shape[0]):
    aMate = dfPurchaseOrderDetail.loc[i]
    if((aMate['is_default'] == 1)):
        dfPurchaseOrderDetail.iloc[[aMate.name], [10]] = aMate['purchase_order_quantity']
    else:
        aProductUnit = dfProductUnitExchange[(dfProductUnitExchange['product_id'] == aMate['product_id']) & (dfProductUnitExchange['result_unit_id'] == aMate['unit_id'])]
        # dfPurchaseOrderDetail.at[aMate.name , 'quantity_exchange'] = round((aMate['purchase_order_quantity']*aProductUnit['source_quantity']) , 4)
        dfPurchaseOrderDetail.iloc[[aMate.name], [10]] = round((aMate['purchase_order_quantity']*aProductUnit['source_quantity']) , 4)

dfPurchaseOrderDetail[dfPurchaseOrderDetail['product_id'] == 6823]

,total_amount,purchase_order_id,is_completed,po_status,product_id,product_unit_id,purchase_order_quantity,is_complete,unit_id,is_default,quantity_exchange
85,650.0,1030,0,approval,6823.0,14683.0,300.0,0.0,5,0,274.32


In [701]:
dfPurchaseOrderDetail = dfPurchaseOrderDetail.merge(dfStockInPurchaseOrder, on = ['purchase_order_id' , 'product_id'], how = 'left').replace(np.NaN , 0)
dfPurchaseOrderDetail['quantity_not_arrived'] = dfPurchaseOrderDetail['quantity_exchange'] - dfPurchaseOrderDetail['quantity_arrived']
dfPurchaseOrderDetail

,total_amount,purchase_order_id,is_completed,po_status,product_id,product_unit_id,purchase_order_quantity,is_complete,unit_id,is_default,quantity_exchange,quantity_arrived,quantity_not_arrived
0,20.0,381,0,approval,3723.0,14234.0,20.0,0.0,5,0,18.288,0.00,18.288
1,11300.0,387,0,approval,749.0,12711.0,100.0,0.0,5,0,91.440,91.44,0.000
2,11300.0,387,0,approval,1127.0,13035.0,2000.0,0.0,5,0,1828.800,1884.20,-55.400
3,11300.0,387,0,approval,1214.0,13118.0,300.0,0.0,5,0,274.320,291.70,-17.380
4,11300.0,387,0,approval,1316.0,13197.0,4000.0,0.0,5,0,3657.600,3759.90,-102.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,9100.0,1375,0,approval,6597.0,6597.0,2500.0,0.0,1,1,2500.000,0.00,2500.000
295,9100.0,1375,0,approval,6598.0,6598.0,500.0,0.0,1,1,500.000,0.00,500.000
296,9100.0,1375,0,approval,6599.0,16697.0,2500.0,0.0,1,1,2500.000,0.00,2500.000
297,9100.0,1375,0,approval,6600.0,6600.0,3500.0,0.0,1,1,3500.000,0.00,3500.000


In [702]:
dfPurchaseOrderDetail[dfPurchaseOrderDetail['product_id'] == 6823]

,total_amount,purchase_order_id,is_completed,po_status,product_id,product_unit_id,purchase_order_quantity,is_complete,unit_id,is_default,quantity_exchange,quantity_arrived,quantity_not_arrived
85,650.0,1030,0,approval,6823.0,14683.0,300.0,0.0,5,0,274.32,300.0,-25.68


In [703]:
dfProductUnitExchange[dfProductUnitExchange['product_id'] == 6823]

,product_id,source_unit_id,result_unit_id,source_quantity,result_quantity
3773,6823,1,5,0.9144,1.0


In [704]:
dfProductUnit[dfProductUnit['product_id'] == 6823]

,product_id,product_unit_id,unit_id,is_default
6973,6823,11168,4,1
10295,6823,14683,5,0


In [705]:
dfPurchaseOrderDetail.loc[(dfPurchaseOrderDetail['quantity_not_arrived'] < 0), 'quantity_not_arrived'] = 0
dfPurchaseOrderDetailSum = dfPurchaseOrderDetail.groupby(['product_id'], as_index=False)['quantity_not_arrived'].sum()
dfPurchaseOrderDetailSum

,product_id,quantity_not_arrived
0,6.0,88.70
1,7.0,88.70
2,8.0,88.70
3,82.0,0.00
4,585.0,200.00
...,...,...
276,11450.0,1051.56
277,11451.0,1051.56
278,11452.0,1000.00
279,11466.0,4.00


In [706]:
dfPurchaseOrderDetailSum[dfPurchaseOrderDetailSum['product_id'] == 6823]

,product_id,quantity_not_arrived
148,6823.0,0.0


#### <input type="checkbox" disabled checked > ĐH GCN `dfOutSourcingDetailSum`

In [732]:
dfOutsourcingStockIn = pd.read_csv('../data_csv/OutsourcingStockIn.csv')
dfOutSourcingDetail = pd.read_csv('../data_csv/OutSourcingDetail.csv')
dfOutSourcingDetail

,name,product_id,outsourcing_order_id,is_complete_order,outsourcing_quantity,is_complete_order_detail,outsourcing_status
0,TTD-051222-254-Trung Việt,86,1405,0,3.5,0,review
1,TTD-051222-254-Trung Việt,639,1405,0,14.0,0,review
2,TTD-051222-254-Trung Việt,9042,1405,0,9.0,0,review
3,TTD-051222-254-Trung Việt,9043,1405,0,52.0,0,review
4,TTD-051222-254-Trung Việt,7371,1405,0,5.7,0,review
...,...,...,...,...,...,...,...
4729,BF-220402-PEMBROKE CLASSIC SPORT NAVY 282C,8418,5,1,63.0,1,completed
4730,AD-220305-REDFILLS PATCH & PATTERN 2 BF-220330...,3983,4,1,5000.0,0,completed
4731,BF-220402-PEMBROKE CLASSIC SPORT GREY,8356,3,1,126.0,0,completed
4732,AD-220305-REDFILLS PATCH & PATTERN 2,8367,2,1,124.0,0,completed


In [733]:
dfOutSourcingDetailNotComplete = dfOutSourcingDetail[(dfOutSourcingDetail['is_complete_order'] != 1) & (dfOutSourcingDetail['is_complete_order_detail'] != 1)
                                                     & (dfOutSourcingDetail['outsourcing_status'] == 'approval')]
dfOutSourcingDetail = dfOutSourcingDetailNotComplete.merge(dfOutsourcingStockIn , on = ['outsourcing_order_id' , 
                                                            'outsourcing_quantity'], how = 'left').replace(np.NaN , 0)
dfOutSourcingDetail

,name,product_id,outsourcing_order_id,is_complete_order,outsourcing_quantity,is_complete_order_detail,outsourcing_status,outsourcing_product_id,is_complete_detail,quantity_stock_in
0,TTD-051222-254-Trung Việt,11433,1404,0,34.0,0,approval,11433,0,0.0
1,TTD-051222-254-Trung Việt,11378,1404,0,319.0,0,approval,11378,0,0.0
2,TTD-051222-254-Trung Việt,11412,1404,0,25.0,0,approval,11412,0,0.0
3,TTD-051222-254-Trung Việt,10832,1404,0,5.7,0,approval,10832,0,0.0
4,TTD-051222-254-Trung Việt,2337,1404,0,6.0,0,approval,2337,0,0.0
...,...,...,...,...,...,...,...,...,...,...
951,SA-PO-38045-Charter Escape Hat,9473,555,0,350.0,0,approval,9473,0,0.0
952,SA-PO-38045-Charter Escape Hat,9471,555,0,5800.0,0,approval,9472,1,5800.0
953,SA-PO-38045-Charter Escape Hat,9471,555,0,5800.0,0,approval,9471,0,0.0
954,SA-PO-38045-Charter Escape Hat,9720,555,0,670.0,0,approval,9720,0,502.0


In [734]:
dfOutSourcingDetail["material_not_in_stock_outSourcing"] = dfOutSourcingDetail["outsourcing_quantity"] - dfOutSourcingDetail['quantity_stock_in']
dfOutSourcingDetail

,name,product_id,outsourcing_order_id,is_complete_order,outsourcing_quantity,is_complete_order_detail,outsourcing_status,outsourcing_product_id,is_complete_detail,quantity_stock_in,material_not_in_stock_outSourcing
0,TTD-051222-254-Trung Việt,11433,1404,0,34.0,0,approval,11433,0,0.0,34.0
1,TTD-051222-254-Trung Việt,11378,1404,0,319.0,0,approval,11378,0,0.0,319.0
2,TTD-051222-254-Trung Việt,11412,1404,0,25.0,0,approval,11412,0,0.0,25.0
3,TTD-051222-254-Trung Việt,10832,1404,0,5.7,0,approval,10832,0,0.0,5.7
4,TTD-051222-254-Trung Việt,2337,1404,0,6.0,0,approval,2337,0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
951,SA-PO-38045-Charter Escape Hat,9473,555,0,350.0,0,approval,9473,0,0.0,350.0
952,SA-PO-38045-Charter Escape Hat,9471,555,0,5800.0,0,approval,9472,1,5800.0,0.0
953,SA-PO-38045-Charter Escape Hat,9471,555,0,5800.0,0,approval,9471,0,0.0,5800.0
954,SA-PO-38045-Charter Escape Hat,9720,555,0,670.0,0,approval,9720,0,502.0,168.0


In [735]:
dfOutSourcingDetail[dfOutSourcingDetail['material_not_in_stock_outSourcing'] < 0] = 0
dfOutSourcingDetailSum = dfOutSourcingDetail.groupby(['product_id'], as_index=False)['material_not_in_stock_outSourcing'].sum()
dfOutSourcingDetailSum

,product_id,material_not_in_stock_outSourcing
0,0,0.0
1,1061,1.5
2,1062,1.0
3,1065,174.5
4,1066,6.5
...,...,...
334,11442,560.0
335,11444,1320.0
336,11446,1320.0
337,11468,4.0


#### <input type="checkbox" disabled checked > PYC nhập kho ( chỉ tính SC) `dfReqStockInProductPrimarySum`

In [736]:
dfReqStockInProductStatus = pd.read_csv("../data_csv/ReqStockInProductStatus.csv")
dfReqStockInProduct = pd.read_csv('../data_csv/ReqStockInProduct.csv')

dfReqStockInProductStatus

,product_id,request_id,quantity_in_stock_by_req
0,3648,6,68.0
1,3896,8,553.0
2,8225,9,50.0
3,5948,9,79.0
4,7564,10,303.0
...,...,...,...
203,6611,127,4100.0
204,11370,128,900.0
205,6591,129,5820.0
206,6737,129,45.0


In [737]:
dfReqStockInProductNotComplete = dfReqStockInProduct[(dfReqStockInProduct['is_complete_req_detail'] != 1) & (dfReqStockInProduct['is_complete_req'] != 1)]
dfReqStockInProductNotComplete

,product_id,request_id,quantity_req_stockin,is_complete_req_detail,is_complete_req
8,249,4,2.0,0,0
19,2448,16,3050.0,0,0
39,1238,24,100.0,0,0
40,6597,26,669.8,0,0
62,7093,36,153.0,0,0
63,3426,36,1.8,0,0
207,3451,111,2.0,0,0


In [725]:
dfReqStockInProductMerge = dfReqStockInProductNotComplete.merge(dfReqStockInProductStatus , on = ['product_id' , 'request_id'] , how = 'left')
dfReqStockInProductPrimary = dfReqStockInProductMerge.merge(dfPrimary[['product_id']] , on = 'product_id' , 
                                                    how = 'inner').sort_values(by = ['product_id']).reset_index(drop= True).replace(np.NaN, 0)
dfReqStockInProductPrimary['material_not_in_stock_request'] = dfReqStockInProductPrimary['quantity_req_stockin'] - dfReqStockInProductPrimary['quantity_in_stock_by_req']
dfReqStockInProductPrimary

,product_id,request_id,quantity_req_stockin,is_complete_req_detail,is_complete_req,quantity_in_stock_by_req,material_not_in_stock_request
0,249,4,2.0,0,0,0.0,2.0
1,2448,16,3050.0,0,0,3000.0,50.0
2,3426,36,1.8,0,0,0.0,1.8
3,3451,111,2.0,0,0,0.0,2.0
4,6597,26,669.8,0,0,0.0,669.8


In [726]:
dfReqStockInProductPrimary[dfReqStockInProductPrimary['material_not_in_stock_request'] < 0] = 0
dfReqStockInProductPrimarySum = dfReqStockInProductPrimary.groupby(['product_id'], as_index=False)['material_not_in_stock_request'].sum()
dfReqStockInProductPrimarySum

,product_id,material_not_in_stock_request
0,249,2.0
1,2448,50.0
2,3426,1.8
3,3451,2.0
4,6597,669.8


#### Tổng đã đặt `dfOrderedQuantity`

In [727]:
orderedQuantity = dfPurchaseOrderDetailSum[['product_id' , 'quantity_not_arrived']].merge(dfOutSourcingDetailSum
                                    [['product_id' , 'material_not_in_stock_outSourcing']], on = 'product_id', how = 'outer').merge(dfReqStockInProductPrimarySum 
                                   [['product_id' , 'material_not_in_stock_request']] ,on = 'product_id', how = 'outer')
                                    
orderedQuantity = orderedQuantity.replace(np.NaN , 0)
orderedQuantity['ordered_quantity'] = orderedQuantity['quantity_not_arrived'] + orderedQuantity[
            'material_not_in_stock_outSourcing'] + orderedQuantity['material_not_in_stock_request']
orderedQuantity = orderedQuantity.sort_values(by = ['product_id']).reset_index(drop = True)
orderedQuantity = orderedQuantity[['product_id' , 'ordered_quantity']]
orderedQuantity
# orderedQuantity[orderedQuantity['product_id'] == 1703]

,product_id,ordered_quantity
0,0.0,0.0
1,6.0,88.7
2,7.0,88.7
3,8.0,88.7
4,82.0,0.0
...,...,...
619,11452.0,1000.0
620,11466.0,4.0
621,11468.0,4.0
622,11469.0,4.0


In [728]:
orderedQuantityRp = dfMaterialReports[['product_id','ordered_quantity']]
compare = orderedQuantity.merge(orderedQuantityRp , on = 'product_id' , how = 'inner')
compare = compare.replace(np.NaN , 0)
compare['quantity_compare'] = round((compare['ordered_quantity_y'] - compare['ordered_quantity_x']) , 4 ) 
compare.loc[compare['quantity_compare'] != 0]

,product_id,ordered_quantity_x,ordered_quantity_y,quantity_compare
18,1065.0,174.5,66.0,-108.5
26,1312.0,126.0,2.0,-124.0
69,2334.0,15.0,1.0,-14.0
102,3983.0,114250.0,57125.0,-57125.0
103,3989.0,114250.0,57125.0,-57125.0
...,...,...,...,...
608,11430.0,326.0,163.0,-163.0
612,11444.0,1320.0,660.0,-660.0
613,11446.0,1320.0,660.0,-660.0
620,11468.0,4.0,2.0,-2.0


## <input type="checkbox" disabled checked >  Đã xuất GCN (Da Xuat - Da Nhap) `outsourcingStockOut`

#### <input type="checkbox" disabled checked > Chi tiết đơn hàng gia công ngoài 
- `outsourcing_detail` NPL GCN nhap ve
- `outsourcing_quota` NPL GCN xuat di

In [1425]:
dfOutSourcingMaterial = pd.read_csv('../data_csv/OutSourcingMaterial.csv')
dfOutSourcingMaterial = dfOutSourcingMaterial.replace(np.NaN , 0)
dfOutSourcingOrder = pd.read_csv('../data_csv/OutSourcingOrder.csv')
dfOutSourcingOrderDetail = dfOutSourcingOrder.merge(dfOutSourcingMaterial, on = ['outsourcing_order_id'] , how = 'left').replace(np.NaN , 0)
dfOutSourcingOrderDetail[dfOutSourcingOrderDetail['product_id'] == 10504]

,name,outsourcing_order_id,is_complete_infos,status,product_id,outsourcing_detail_id,outsourcing_product_id,quantity_outsourcing_detail,is_complete_detail,total_quota,is_exported,quota_percent
3936,TTDKTY-220914-BK,898,0,approval,10504,3959,10709,740.0,0,17.83,1,0.0241
3937,TTDKTY-220914-BK,898,0,approval,10504,3960,10753,1200.0,1,33.00,1,0.0275
6561,TTDSTK-221121-BK-Buttercream-Blank,1388,0,approval,10504,6097,10753,330.0,0,9.07,1,0.0275
6562,TTDSTK-221121-BK-Buttercream-Blank,1388,0,approval,10504,6098,10709,200.0,0,4.82,1,0.0241


In [1396]:
# productNotComplete = dfOutSourcingOrderDetail[(dfOutSourcingOrderDetail['is_complete_infos'] != 1 )
#                                               & (dfOutSourcingOrderDetail['status'] == 'approval')].drop_duplicates(subset=['outsourcing_order_id'], keep='first')['outsourcing_order_id'].to_list()

dfOutSourcingOrderDetail = dfOutSourcingOrderDetail[(dfOutSourcingOrderDetail['is_complete_infos'] != 1 )
                                              & (dfOutSourcingOrderDetail['status'] == 'approval')]
productNotComplete = dfOutSourcingOrderDetail[((dfOutSourcingOrderDetail['is_complete_detail'] == 0 )
                                              | (dfOutSourcingOrderDetail['is_exported'] == 0))]

productNotComplete.head(5)
productNotComplete[productNotComplete['product_id'] == 10504]

,name,outsourcing_order_id,is_complete_infos,status,product_id,outsourcing_detail_id,outsourcing_product_id,quantity_outsourcing_detail,is_complete_detail,total_quota,is_exported,quota_percent
3936,TTDKTY-220914-BK,898,0,approval,10504,3959,10709,740.0,0,17.83,1,0.0241
6561,TTDSTK-221121-BK-Buttercream-Blank,1388,0,approval,10504,6097,10753,330.0,0,9.07,1,0.0275
6562,TTDSTK-221121-BK-Buttercream-Blank,1388,0,approval,10504,6098,10709,200.0,0,4.82,1,0.0241


In [1397]:
# df = dfOutSourcingStockMerge[dfOutSourcingStockMerge['outsourcing_order_id'].isin(productNotComplete)].reset_index(drop = True)

#### <input type="checkbox" disabled checked > Số lượng nguyên phụ liệu xuất thực tế đi GCN

In [1428]:
dfOutSourcingStockOut = pd.read_csv('../data_csv/OutSourcingStockOut.csv')
dfOutSourcingStockOut = dfOutSourcingStockOut.replace(np.NaN , 0)
dfOutSourcingStockOut

dfOutSourcingStockOut[dfOutSourcingStockOut['product_id'] == 10504]

,product_id,outsourcing_order_id,quantity_stock_out,is_exported,outsourcing_detail_id,assign_product_stockout_id,quantity_exported_outsourcing
6879,10504,1388,9.07,1,6097,50885.0,7.78
6880,10504,1388,9.07,1,6097,50886.0,1.29
6881,10504,1388,4.82,1,6098,50887.0,4.82
6882,10504,898,17.83,1,3959,33015.0,17.83
6883,10504,898,33.00,1,3960,33016.0,23.58
6884,10504,898,33.00,1,3960,33017.0,6.90


In [1420]:
dfOutSourcingStockOut = pd.read_csv('../data_csv/OutSourcingStockOut.csv')
dfOutSourcingStockOut = dfOutSourcingStockOut.replace(np.NaN , 0)
dfNotComplete = dfOutSourcingStockOut.merge(productNotComplete , on = ['outsourcing_order_id' , 'is_exported' , 'product_id' , 'outsourcing_detail_id'] , how = 'right')
dfNotComplete = dfNotComplete.merge(dfRelationship , on = ['outsourcing_product_id' , 'product_id'] , how = 'inner')
# dfNotComplete = dfNotComplete[dfNotComplete['is_exported'] == 0]
# dfNotComplete = dfOutSourcingStockOut
dfNotComplete.head(2)

,product_id,outsourcing_order_id,quantity_stock_out,is_exported,outsourcing_detail_id,assign_product_stockout_id,quantity_exported_outsourcing,name,is_complete_infos,status,outsourcing_product_id,quantity_outsourcing_detail,is_complete_detail,total_quota,quota_percent,product_code,quota_outsourcing
0,9473,555,350.00,0,2549,0.0,0.0,SA-PO-38045-Charter Escape Hat,0,approval,9474,350.0,0,350.00,1.0000,9-PH0757-BLK-00-12,1.0000
1,1537,555,35.59,1,2550,20131.0,41.0,SA-PO-38045-Charter Escape Hat,0,approval,9473,350.0,0,35.59,0.1017,1-PH0757-BLK-00-UL,0.1017


In [1437]:
dfNotComplete[dfNotComplete['product_id'] == 10504]

,product_id,outsourcing_order_id,quantity_stock_out,is_exported,outsourcing_detail_id,assign_product_stockout_id,quantity_exported_outsourcing,name,is_complete_infos,status,outsourcing_product_id,quantity_outsourcing_detail,is_complete_detail,total_quota,quota_percent,product_code,quota_outsourcing
62,10504,898,17.83,1,3959,33015.0,17.83,TTDKTY-220914-BK,0,approval,10709,740.0,0,17.83,0.0241,1-P2585Q-BTC-L3-EV,0.0241
63,10504,1388,4.82,1,6098,50887.0,4.82,TTDSTK-221121-BK-Buttercream-Blank,0,approval,10709,200.0,0,4.82,0.0241,1-P2585Q-BTC-L3-EV,0.0241
503,10504,1388,9.07,1,6097,50885.0,7.78,TTDSTK-221121-BK-Buttercream-Blank,0,approval,10753,330.0,0,9.07,0.0275,1-P2585Q-BTC-L3-EV,0.0275
504,10504,1388,9.07,1,6097,50886.0,1.29,TTDSTK-221121-BK-Buttercream-Blank,0,approval,10753,330.0,0,9.07,0.0275,1-P2585Q-BTC-L3-EV,0.0275


In [1438]:
dfRelationship[dfRelationship['product_id'] == 10504]

,product_id,outsourcing_product_id,product_code,quota_outsourcing
4232,10504,10708,1-P2585Q-BTC-L3-EV,0.0310
4234,10504,10709,1-P2585Q-BTC-L3-EV,0.0241
4275,10504,10753,1-P2585Q-BTC-L3-EV,0.0275


In [1414]:
dfNotComplete[dfNotComplete['outsourcing_order_id'] == 10504]

,product_id,outsourcing_order_id,quantity_stock_out,is_exported,outsourcing_detail_id,assign_product_stockout_id,quantity_exported_outsourcing,name,is_complete_infos,status,outsourcing_product_id,quantity_outsourcing_detail,is_complete_detail,total_quota,quota_percent,product_code,quota_outsourcing


In [1415]:
dfOutSourcingStockOutSum = dfNotComplete.groupby(['product_id'], as_index=False)['quantity_exported_outsourcing'].sum()
dfOutSourcingStockOutSum

,product_id,quantity_exported_outsourcing
0,1,233.50
1,5,355.84
2,66,1.61
3,69,4.03
4,74,4.00
...,...,...
283,11339,13.40
284,11340,0.00
285,11355,378.00
286,11445,45.00


In [1416]:
dfOutSourcingStockOutSum[dfOutSourcingStockOutSum['product_id'] == 10504]

,product_id,quantity_exported_outsourcing
252,10504,31.72


#### Số lượng nguyên phụ liệu nhập về từ GCN

In [1434]:
dfOutSourcingStockIn = pd.read_csv('../data_csv/OutsourcingStockIn.csv')
dfOutSourcingStockIn = dfOutSourcingStockIn.replace(np.NaN , 0)
dfOutSourcingStockIn

,outsourcing_order_id,outsourcing_product_id,is_complete_detail,outsourcing_quantity,quantity_stock_in
0,1405,86,0,3.5,0.0
1,1405,639,0,14.0,0.0
2,1405,9042,0,9.0,0.0
3,1405,9043,0,52.0,0.0
4,1405,7371,0,5.7,0.0
...,...,...,...,...,...
5852,5,8418,1,63.0,63.0
5853,4,3983,0,5000.0,0.0
5854,3,8356,0,126.0,113.0
5855,2,8367,0,124.0,0.0


In [1436]:
dfOutSourcingStockMerge = dfOutSourcingStockIn.merge(dfRelationship , on = 'outsourcing_product_id' , how = 'left')
dfOutSourcingStockMerge['convert_total_quota'] = dfOutSourcingStockMerge['quantity_stock_in'] * dfOutSourcingStockMerge['quota_outsourcing']

dfOutSourcingStockMerge[dfOutSourcingStockMerge['product_id'] == 10504]

,outsourcing_order_id,outsourcing_product_id,is_complete_detail,outsourcing_quantity,quantity_stock_in,product_id,product_code,quota_outsourcing,convert_total_quota
108,1388,10753,0,330.0,0.0,10504.0,1-P2585Q-BTC-L3-EV,0.0275,0.0000
109,1388,10709,0,200.0,0.0,10504.0,1-P2585Q-BTC-L3-EV,0.0241,0.0000
3946,898,10709,0,740.0,588.0,10504.0,1-P2585Q-BTC-L3-EV,0.0241,14.1708
3947,898,10709,0,740.0,136.0,10504.0,1-P2585Q-BTC-L3-EV,0.0241,3.2776
3948,898,10753,1,1200.0,1200.0,10504.0,1-P2585Q-BTC-L3-EV,0.0275,33.0000


In [1410]:
dfOutSourcingStockIn = pd.read_csv('../data_csv/OutsourcingStockIn.csv')
dfOutSourcingStockIn = dfOutSourcingStockIn.replace(np.NaN , 0)
dfNotComplete = dfOutSourcingStockIn
dfNotComplete = dfOutSourcingStockIn.merge(productNotComplete , on = ['outsourcing_order_id' , 'is_complete_detail' , 'outsourcing_product_id'] , how = 'right')
dfNotComplete = dfNotComplete.groupby(['product_id','outsourcing_product_id' , 'outsourcing_order_id' , 'is_complete_detail','outsourcing_quantity' , 'quota_percent'], as_index=False)['quantity_stock_in'].sum()

dfNotComplete = dfNotComplete.drop(columns= {'product_id' , 'quota_percent'}).drop_duplicates(subset = ['outsourcing_order_id' , 'outsourcing_product_id' , 'quantity_stock_in'] , keep = 'first')
dfOutSourcingStockMerge = dfNotComplete.merge(dfRelationship , on = 'outsourcing_product_id' , how = 'left')
dfOutSourcingStockMerge['convert_total_quota'] = dfOutSourcingStockMerge['quantity_stock_in'] * dfOutSourcingStockMerge['quota_outsourcing']
dfOutSourcingStockMerge

dfOutSourcingStockMerge[dfOutSourcingStockMerge['product_id'] == 10504]

,outsourcing_product_id,outsourcing_order_id,is_complete_detail,outsourcing_quantity,quantity_stock_in,product_id,product_code,quota_outsourcing,convert_total_quota
586,10709,898,0,740.0,724.0,10504,1-P2585Q-BTC-L3-EV,0.0241,17.4484
587,10709,1388,0,200.0,0.0,10504,1-P2585Q-BTC-L3-EV,0.0241,0.0000
588,10753,1388,0,330.0,0.0,10504,1-P2585Q-BTC-L3-EV,0.0275,0.0000


In [1408]:
dfOutSourcingStockInSum = dfOutSourcingStockMerge.groupby(['product_id'], as_index=False)['convert_total_quota'].sum()
dfOutSourcingStockInSum

,product_id,convert_total_quota
0,1,255.200
1,2,35.000
2,5,0.000
3,66,0.876
4,69,0.000
...,...,...
295,11339,0.000
296,11340,172.000
297,11355,0.000
298,11445,0.000


In [1409]:
dfOutSourcingStockInSum[dfOutSourcingStockInSum['product_id'] == 10504]

,product_id,convert_total_quota
262,10504,17.4484


#### <input type="checkbox" disabled checked > Chung tu nhap kho

In [1383]:
# dfOutSourcingMaterialAllNotComplete

In [1384]:
# dfOutSourcingMaterialAllNotComplete[dfOutSourcingMaterialAllNotComplete['outsourcing_product_id'] == 8422]

In [1385]:
# dfImportOutSourcing = pd.read_csv('data_csv/ImportOutSourcing.csv')
# dfImportOutSourcing = dfImportOutSourcing.rename(columns= {'product_id' : 'outsourcing_product_id'})
# dfImportOutSourcingMerge = dfImportOutSourcing.merge(dfRelationship , on = 'outsourcing_product_id' , how = 'inner')
# dfImportOutSourcingMerge

In [1386]:
# dfImportOutSourcingMergeNotComplete =   dfImportOutSourcingMerge.merge(dfOutSourcingMaterialAllNotComplete , on = ['outsourcing_product_id' , 'outsourcing_order_id'] , how = 'right')
# dfImportOutSourcingMergeNotComplete

In [1387]:
# df = dfImportOutSourcingMerge[dfImportOutSourcingMerge['outsourcing_order_id'].isin(dfOutSourcingMaterialAllNotComplete['purchase_order_id'].to_list())].reset_index(drop = True)

#### outSourcingStockOut

In [1388]:
dfOutSourcingStockInSum

,product_id,convert_total_quota
0,1,255.200
1,2,35.000
2,5,0.000
3,66,0.876
4,69,0.000
...,...,...
295,11339,0.000
296,11340,172.000
297,11355,0.000
298,11445,0.000


In [1389]:
outsourcingStockOut = dfOutSourcingStockInSum.merge(dfOutSourcingStockOutSum , on = 'product_id' , how = 'outer')
outsourcingStockOut = outsourcingStockOut.replace(np.NaN , 0)
outsourcingStockOut['outsourcing_stock_out'] = round(outsourcingStockOut['quantity_exported_outsourcing'] - outsourcingStockOut['convert_total_quota'] , 5)
outsourcingStockOut[outsourcingStockOut['product_id'] == 594]
outsourcingStockOut = outsourcingStockOut[['product_id' , 'outsourcing_stock_out']]
outsourcingStockOut.loc[(outsourcingStockOut['outsourcing_stock_out'] < 0), 'outsourcing_stock_out'] = 0
outsourcingStockOut.head(10)


,product_id,outsourcing_stock_out
0,1,0.0000
1,2,0.0000
2,5,355.8400
3,66,0.7340
4,69,4.0300
5,74,4.0000
6,82,1.7188
7,96,50.9100
8,102,61.2000
9,107,11.0400


In [1390]:
outsourcingStockOutRp = dfMaterialReports[['product_id','outsourcing_stock_out']]
outsourcingStockOutRp

/home/tranthang/.local/lib/python3.10/site-packages/IPython/core/displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,product_id,outsourcing_stock_out
0,1,0.00
1,2,0.00
2,3,0.00
3,4,0.00
4,5,355.84
...,...,...
11329,11468,0.00
11330,11469,0.00
11331,11470,0.00
11332,11471,0.00


In [1391]:
outsourcingStockOutRp = dfMaterialReports[['product_id','outsourcing_stock_out']]
compare = outsourcingStockOut.merge(outsourcingStockOutRp , on = 'product_id' , how = 'outer')
compare = compare.replace(np.NaN , 0)
compare['quantity_compare'] = round((compare['outsourcing_stock_out_y'] - compare['outsourcing_stock_out_x']) , 5) 
# compare[compare['product_id'] == 594]
compare.loc[compare['quantity_compare'] != 0]

,product_id,outsourcing_stock_out_x,outsourcing_stock_out_y,quantity_compare
137,1785,117.0832,117.5900,0.5068
262,10504,14.2716,11.7516,-2.5200


In [1393]:
compare[compare['product_id'] == 10504]

,product_id,outsourcing_stock_out_x,outsourcing_stock_out_y,quantity_compare
262,10504,14.2716,11.7516,-2.52


## <input type="checkbox" disabled checked > Tồn kho `inStock`

#### <input type="checkbox" disabled checked > Tồn kho theo material_reports

In [324]:
dfProduct = pd.read_csv("../data_csv/Product.csv")

#### <input type="checkbox" disabled checked > Tồn kho theo NK - XK

##### <input type="checkbox" disabled checked > Nhập kho

In [125]:
dfProductStockInRealDetails.head(10)

,product_id,quantity_in_stock
0,1,5120.6
1,1,4937.8
2,1,4572.0
3,1,4023.3
4,1,548.6
5,1,4572.0
6,1,4572.0
7,2,4572.0
8,2,7680.9
9,2,5486.4


In [126]:
dfProductStockInRealDetailsDropDup = dfProductStockInRealDetails.drop_duplicates(subset=['product_id'], keep='first').reset_index(drop= True)
dfProductStockInRealDetailsGroupbySum = dfProductStockInRealDetails.groupby(['product_id'], as_index=False)['quantity_in_stock'].sum()
dfProductStockInRealDetailsGroupbySum.sort_values(by = ['product_id'])
dfProductStockInRealDetailsGroupbySum

,product_id,quantity_in_stock
0,1,28346.3
1,2,78455.3
2,3,5303.5
3,4,4663.4
4,5,13167.3
...,...,...
4109,11465,2.0
4110,11466,6.0
4111,11467,1.0
4112,11470,6.0


##### <input type="checkbox" disabled checked > Xuất kho

In [127]:
dfProductionStockOut

,product_id,quantity_stock_out
0,1,35.0
1,1,15.0
2,1,3.0
3,1,64.0
4,1,5.5
...,...,...
51460,11441,12000.0
51461,11445,140.0
51462,11445,45.0
51463,11454,6.8


In [128]:
dfProductionStockOut[dfProductionStockOut['product_id'] == 2]['quantity_stock_out'].sum()

54857.020000000004

In [129]:
dfProductionStockOutDropDup = dfProductionStockOut.drop_duplicates(subset=['product_id'], keep='first').reset_index(drop= True)
dfProductionStockOutGroupbySum = dfProductionStockOut.groupby(['product_id'], as_index=False)['quantity_stock_out'].sum()
dfProductionStockOutGroupbySum.sort_values(by = ['product_id'])
dfProductionStockOutGroupbySum

,product_id,quantity_stock_out
0,1,21678.39
1,2,54857.02
2,3,865.80
3,4,7702.50
4,5,18160.22
...,...,...
4800,11416,5.00
4801,11441,20000.00
4802,11445,185.00
4803,11454,6.80


##### <input type="checkbox" disabled checked > Điều chỉnh tăng giảm kho

In [130]:
dfWhCorrectionDetail = pd.read_csv('../data_csv/WhCorrectionDetail.csv')
dfWhCorrectionDetail

,quantity,product_id,is_increase
0,366.0,1,0
1,7465.6,2,1
2,7465.6,2,1
3,7465.6,2,0
4,4725.0,2,0
...,...,...,...
6087,100.0,10681,1
6088,160.0,10682,1
6089,110.0,10683,1
6090,1155.0,11012,1


In [131]:
dfWhCorrectionDetail['is_increase'] = dfWhCorrectionDetail['is_increase'].replace(0 , -1)
dfWhCorrectionDetail['quantity_correction'] = dfWhCorrectionDetail['quantity'] * dfWhCorrectionDetail['is_increase']
dfWhCorrectionDetail

,quantity,product_id,is_increase,quantity_correction
0,366.0,1,-1,-366.0
1,7465.6,2,1,7465.6
2,7465.6,2,1,7465.6
3,7465.6,2,-1,-7465.6
4,4725.0,2,-1,-4725.0
...,...,...,...,...
6087,100.0,10681,1,100.0
6088,160.0,10682,1,160.0
6089,110.0,10683,1,110.0
6090,1155.0,11012,1,1155.0


In [132]:
dfWhCorrectionDetail[dfWhCorrectionDetail['product_id'] == 4]

,quantity,product_id,is_increase,quantity_correction
18,8230.0,4,1,8230.0
19,1920.0,4,-1,-1920.0


In [133]:
# dfWhCorrectionDetailDropDup = dfWhCorrectionDetail.drop_duplicates(subset=['product_id'], keep='first').reset_index(drop= True)
dfWhCorrectionDetailGroupbySum = dfWhCorrectionDetail.groupby(['product_id'], as_index=False)['quantity_correction'].sum().sort_values(by = ['product_id'])
dfWhCorrectionDetailGroupbySum

,product_id,quantity_correction
0,1,-366.00
1,2,-9855.55
2,4,6310.00
3,5,7317.00
4,6,94.00
...,...,...
3870,10681,100.00
3871,10682,160.00
3872,10683,110.00
3873,11012,1155.00


In [134]:
inStock = pd.merge(pd.merge(dfProductStockInRealDetailsGroupbySum, dfProductionStockOutGroupbySum , on = 
                'product_id', how = 'outer' ), dfWhCorrectionDetailGroupbySum , on =  
                'product_id' , how = 'outer').sort_values(by = ['product_id']).replace(np.NaN , 0).reset_index(drop = True)
inStock 

,product_id,quantity_in_stock,quantity_stock_out,quantity_correction
0,1,28346.3,21678.39,-366.00
1,2,78455.3,54857.02,-9855.55
2,3,5303.5,865.80,0.00
3,4,4663.4,7702.50,6310.00
4,5,13167.3,18160.22,7317.00
...,...,...,...,...
6758,11465,2.0,0.00,0.00
6759,11466,6.0,0.00,0.00
6760,11467,1.0,0.00,0.00
6761,11470,6.0,0.00,0.00


#### Tồn kho `inStock`

In [135]:
inStock['in_stock'] = round(inStock['quantity_in_stock'] - inStock['quantity_stock_out'] + inStock['quantity_correction'], 4)
inStock = inStock[['product_id' , 'in_stock']]
inStock

,product_id,in_stock
0,1,6301.91
1,2,13742.73
2,3,4437.70
3,4,3270.90
4,5,2324.08
...,...,...
6758,11465,2.00
6759,11466,6.00
6760,11467,1.00
6761,11470,6.00


#### Check

In [136]:
inStockRp = dfMaterialReports[['product_id','in_stock']]
compare = inStock.merge(inStockRp , on = 'product_id' , how = 'inner')
compare = compare.replace(np.NaN , 0)
compare['quantity_compare'] = round((compare['in_stock_y'] - compare['in_stock_x']) , 4 ) 
compare.loc[compare['quantity_compare'] != 0]

,product_id,in_stock_x,in_stock_y,quantity_compare


## <input type="checkbox" disabled checked > Tính số cần

In [1232]:
dfOrder = pd.read_csv("../data_csv/Order.csv")
dfOrder

,product_id,order_id,product_code,order_status,position,total_quota,complete_order,booking_status
0,77,1,1-CVAC04-SCA-L2-HV,completed,Mặt phải,6.5625,1,approval
1,124,1,1-CVAC12-ROL-L2-IW,completed,Mặt trái,6.5625,1,approval
2,74,1,1-CVAC04-SCA-00-HV,completed,Hông phải+nút,6.7200,1,approval
3,122,1,1-CVAC12-ROL-00-IW,completed,Hông trái,6.4000,1,approval
4,74,1,1-CVAC04-SCA-00-HV,completed,Đuôi phải,7.8280,1,approval
...,...,...,...,...,...,...,...,...
55960,5948,2518,7-CM0BLK-BLK-50-PP,approval,chỉ may black,12.0000,0,review
55961,8916,2518,9-GL2608-450-00-TA,approval,Giấy lót nón,150.0000,0,review
55962,6731,2518,9-B03680-WHI-00-HN,approval,Bao nylon,6.0000,0,review
55963,8464,2518,9-554331-Y00-5L-BN,approval,Thùng lớn,2.0000,0,review


In [138]:
dfRelationship = pd.read_csv('../data_csv/Relationship.csv')

In [139]:
dfProduct = pd.read_csv("../data_csv/Product.csv")

### <input type="checkbox" disabled checked > 1. Cần - XKTT
- Đơn hàng bán(múi + SA)
- PYC Xuất kho
- YC xuất bù LSX
- Xuất đổi trả(chỉ tính sơ cấp)

In [169]:
dfRelationship

,product_id,outsourcing_product_id,product_code,quota_outsourcing
0,73,75,1-CVAC04-SCA-00-IW,1.0
1,73,78,1-CVAC04-SCA-00-IW,1.0
2,73,79,1-CVAC04-SCA-00-IW,1.0
3,80,81,1-CVAC05-DBN-00-IW,1.0
4,96,101,1-CVAC07-GRY-00-HV,1.0
...,...,...,...,...
4826,11355,11412,1-P2585Q-CTG-00-EV,1.0
4827,8311,11412,1-GOLDAI-WHI-00-EP,1.0
4828,5,11412,1-LUOIMA-WHI-28-CM,1.0
4829,777,11468,1-NY0000-BLK-00-PE,1.0


#### <input type="checkbox" disabled checked > 1.1 Cần - XKTT + Đơn hàng bán(múi + SA) `needQuantityOrder`

In [170]:
dfOrder[dfOrder['product_id'] == 8508]

,product_id,order_id,product_code,order_status,position,total_quota,complete_order,booking_status
670,8508,36,4-KATE00-BLK-N0-28,completed,Viền,744.00,1,approval
696,8508,37,4-KATE00-BLK-N0-28,completed,Viền,447.00,1,approval
724,8508,38,4-KATE00-BLK-N0-28,completed,Viền,744.00,1,approval
749,8508,39,4-KATE00-BLK-N0-28,completed,Viền,447.00,1,approval
778,8508,40,4-KATE00-BLK-N0-28,completed,Viền,447.00,1,approval
...,...,...,...,...,...,...,...,...
55819,8508,2511,4-KATE00-BLK-N0-28,approval,Viền trong chóp,449.40,0,review
55838,8508,2512,4-KATE00-BLK-N0-28,approval,Viền trong chóp,5.45,0,review
55853,8508,2513,4-KATE00-BLK-N0-28,approval,Viền,1390.50,0,approval
55893,8508,2515,4-KATE00-BLK-N0-28,approval,Viền,148.50,0,approval


In [171]:
dfProductionOrder                   = pd.read_csv('../data_csv/ProductionOrder.csv') 
dfProductionDetailStockOut          = pd.read_csv('../data_csv/ProductionDetailStockOut.csv')
dfProductionDetailStockOut = dfProductionDetailStockOut.groupby(['product_id', 'order_id' , 'production_order_id' , 'complete_prodtion_stockout' , 'is_exported' , 'quantity_production_order' ], as_index=False)['quantity_exported_production_order'].sum()
dfProductionDetailStockOut          = dfProductionDetailStockOut.merge(dfProductionOrder, on = 'production_order_id' , how = 'inner')

dfProductionDetailStockOut = dfProductionDetailStockOut.groupby(['product_id', 'order_id' , 'complete_prodtion_stockout' , 'is_exported'], as_index=False)[ 'quantity_production_order' , 'quantity_exported_production_order'].sum()

dfProductionDetailStockOut[dfProductionDetailStockOut['product_id'] == 8508]

/tmp/ipykernel_8714/3602039158.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfProductionDetailStockOut = dfProductionDetailStockOut.groupby(['product_id', 'order_id' , 'complete_prodtion_stockout' , 'is_exported'], as_index=False)[ 'quantity_production_order' , 'quantity_exported_production_order'].sum()


,product_id,order_id,complete_prodtion_stockout,is_exported,quantity_production_order,quantity_exported_production_order
31245,8508,36,0,1,744.00,744.0
31246,8508,37,0,1,447.00,447.0
31247,8508,38,0,1,744.00,744.0
31248,8508,39,0,1,447.00,447.0
31249,8508,40,0,1,447.00,447.0
...,...,...,...,...,...,...
31939,8508,2483,0,0,148.50,0.0
31940,8508,2492,0,0,290.34,0.0
31941,8508,2502,0,0,207.35,0.0
31942,8508,2513,0,0,1390.50,0.0


In [172]:
dfOrder                             = pd.read_csv('../data_csv/Order.csv')
dfProductionOrder                   = pd.read_csv('../data_csv/ProductionOrder.csv') 
dfProductionDetailStockOut          = pd.read_csv('../data_csv/ProductionDetailStockOut.csv')
dfProductionDetailStockOut = dfProductionDetailStockOut.groupby(['product_id', 'order_id' , 'production_order_id' , 'complete_prodtion_stockout' , 'is_exported' , 'quantity_production_order' ], as_index=False)['quantity_exported_production_order'].sum()
dfProductionDetailStockOut = dfProductionDetailStockOut.groupby(['product_id', 'order_id' , 'complete_prodtion_stockout' , 'is_exported'], as_index=False)[ 'quantity_production_order' , 'quantity_exported_production_order'].sum()

# dfProductionDetailStockOut          = dfProductionDetailStockOut.merge(dfProductionOrder, on = 'production_order_id' , how = 'inner')
# dfProductionDetailStockOutNotComplete  = dfProductionDetailStockOut[(dfProductionDetailStockOut['production_status'] != 'completed') 
#                                                                     & (dfProductionDetailStockOut['is_complete'] != 1)]

dfOrderNotCompleted                 = dfOrder[(dfOrder['order_status'] != 'completed') & (dfOrder['booking_status'] != 'review')]
dfGroup = dfProductionDetailStockOut.replace(np.NaN , 0).groupby(['product_id', 'order_id', 'complete_prodtion_stockout' , 'is_exported', 
                        'quantity_production_order'], as_index=False)['quantity_exported_production_order'].sum()

dfProductionDetailStockOutMerge     = dfOrderNotCompleted.merge(dfGroup , on = ['product_id' , 'order_id'], how = 'left').replace(np.NaN , 0)
dfProductionDetailStockOutMerge[dfProductionDetailStockOutMerge['product_id'] == 8508]

/tmp/ipykernel_8714/2632235245.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfProductionDetailStockOut = dfProductionDetailStockOut.groupby(['product_id', 'order_id' , 'complete_prodtion_stockout' , 'is_exported'], as_index=False)[ 'quantity_production_order' , 'quantity_exported_production_order'].sum()


,product_id,order_id,product_code,order_status,position,total_quota,complete_order,booking_status,complete_prodtion_stockout,is_exported,quantity_production_order,quantity_exported_production_order
793,8508,855,4-KATE00-BLK-N0-28,approval,Viền bọc đuôi,2404.00,0,approval,0.0,1.0,2404.00,2404.00
879,8508,857,4-KATE00-BLK-N0-28,approval,Viền bọc đuôi,1600.20,0,approval,0.0,1.0,1600.20,1600.20
1145,8508,886,4-KATE00-BLK-N0-28,approval,Viền trong,14002.80,0,approval,0.0,0.0,0.00,0.00
1190,8508,887,4-KATE00-BLK-N0-28,approval,Viền trong,13162.80,0,approval,0.0,0.0,0.00,0.00
5451,8508,1340,4-KATE00-BLK-N0-28,approval,Viền 2.8cm trơn,370.99,0,approval,0.0,1.0,370.99,370.99
...,...,...,...,...,...,...,...,...,...,...,...,...
17566,8508,2492,4-KATE00-BLK-N0-28,approval,Viền,290.34,0,approval,0.0,0.0,290.34,0.00
17822,8508,2502,4-KATE00-BLK-N0-28,approval,"Dây viền lớp ngoài, nối vành",47.85,0,approval,0.0,0.0,207.35,0.00
17823,8508,2502,4-KATE00-BLK-N0-28,approval,Dây viền lớp trong,159.50,0,approval,0.0,0.0,207.35,0.00
17945,8508,2513,4-KATE00-BLK-N0-28,approval,Viền,1390.50,0,approval,0.0,0.0,1390.50,0.00


In [173]:
temp = dfProductionDetailStockOutMerge.groupby(['product_id', 'order_id' , 'product_code', 'quantity_production_order', 'quantity_exported_production_order' ], as_index=False)['total_quota'].sum()
temp1 = temp.groupby(['product_id', 'order_id' , 'product_code', 'total_quota' ], as_index=False)['quantity_production_order' , 'quantity_exported_production_order'].sum()
temp1['need_quantity_order'] = temp1['total_quota'] - temp1['quantity_exported_production_order']
needQuantityOrder = temp1.groupby(['product_id'], as_index=False)['total_quota' , 'quantity_production_order', 'quantity_exported_production_order' , 'need_quantity_order'].sum()
needQuantityOrder[(needQuantityOrder['product_id'] == 8508)]

/tmp/ipykernel_8714/2125250399.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp1 = temp.groupby(['product_id', 'order_id' , 'product_code', 'total_quota' ], as_index=False)['quantity_production_order' , 'quantity_exported_production_order'].sum()
/tmp/ipykernel_8714/2125250399.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  needQuantityOrder = temp1.groupby(['product_id'], as_index=False)['total_quota' , 'quantity_production_order', 'quantity_exported_production_order' , 'need_quantity_order'].sum()


,product_id,total_quota,quantity_production_order,quantity_exported_production_order,need_quantity_order
1371,8508,126714.1,85404.22,83219.03,43495.07


In [174]:
needQuantityOrder['need_quantity_order'] = round((needQuantityOrder['total_quota'] - needQuantityOrder['quantity_exported_production_order']) , 4 )
needQuantityOrder.loc[(needQuantityOrder['need_quantity_order'] < 0), 'need_quantity_order'] = 0
needQuantityOrder.head(20)

,product_id,total_quota,quantity_production_order,quantity_exported_production_order,need_quantity_order
0,1,118.4100,0.0000,0.0000,118.41
1,2,6945.7500,2488.0900,2561.2900,4384.46
2,6,51.0000,0.0000,0.0000,51.00
3,7,60.4600,0.0000,0.0000,60.46
4,8,69.3800,0.0000,0.0000,69.38
5,9,35.8100,0.0000,0.0000,35.81
6,10,28.6000,0.0000,0.0000,28.60
7,12,62.1500,0.0000,0.0000,62.15
8,13,57.0000,40.2000,40.2000,16.80
9,15,4.4500,0.0000,0.0000,4.45


In [175]:
needQuantityOrder[needQuantityOrder['product_id'] == 1199]

,product_id,total_quota,quantity_production_order,quantity_exported_production_order,need_quantity_order
194,1199,10.48,0.0,0.0,10.48


#### <input type="checkbox" disabled checked > 1.2 Cần - XKTT + PYC Xuất kho `needQuantityReqStockOut`

In [176]:
# dfReqStockoutProductStockOutMerge[dfReqStockoutProductStockOutMerge['req_stockout_status'].isna()]

In [177]:
dfReqStockoutProduct = pd.read_csv('../data_csv/ReqStockoutProduct.csv')
dfReqStockoutProductStockOut = pd.read_csv('../data_csv/ReqStockoutProductStockOut.csv')
dfReqStockoutProductStockOut[dfReqStockoutProductStockOut['product_id'] == 10667]

,product_id,request_id,is_exported,is_complete_req_detail,quantity_req_stockout,assign_product_stockout_id,quantity_exported_req_stockout
6387,10667,1958,1,1,0.5,49987.0,0.5


In [178]:
dfReqStockoutProduct = pd.read_csv('../data_csv/ReqStockoutProduct.csv')
dfReqStockoutProductStockOut = pd.read_csv('../data_csv/ReqStockoutProductStockOut.csv')
dfReqStockoutProductStockOut = dfReqStockoutProductStockOut.replace(np.NaN , 0)

dfReqStockoutProductStockOut = dfReqStockoutProductStockOut.groupby(['product_id', 'request_id', 'is_exported', 
                                    'is_complete_req_detail','quantity_req_stockout'], as_index=False)['quantity_exported_req_stockout'].sum()
dfReqStockoutProductStockOut
dfReqStockoutProductStockOutMerge           = dfReqStockoutProductStockOut.merge(dfReqStockoutProduct, on = 'request_id' , how = 'right')

dfProductionDetailStockOutNotComplete       = dfReqStockoutProductStockOutMerge[(dfReqStockoutProductStockOutMerge[
                                        'req_stockout_status'] != 'completed') & (dfReqStockoutProductStockOutMerge[
                                        'is_complete_req_detail'] != 1) & (dfReqStockoutProductStockOutMerge[
                                        'req_stockout_status'] != 'review') ]

needQuantityReqStockOut = dfProductionDetailStockOutNotComplete.groupby(['product_id'], as_index=False)['quantity_req_stockout' , 'quantity_exported_req_stockout'].sum()
needQuantityReqStockOut[needQuantityReqStockOut['product_id'] == 10667]

/tmp/ipykernel_8714/1695795973.py:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  needQuantityReqStockOut = dfProductionDetailStockOutNotComplete.groupby(['product_id'], as_index=False)['quantity_req_stockout' , 'quantity_exported_req_stockout'].sum()


,product_id,quantity_req_stockout,quantity_exported_req_stockout


In [179]:
needQuantityReqStockOut['need_quantity_req_stock_out'] = needQuantityReqStockOut['quantity_req_stockout'] - needQuantityReqStockOut['quantity_exported_req_stockout'] 
needQuantityReqStockOut.loc[(needQuantityReqStockOut['need_quantity_req_stock_out'] < 0), 'need_quantity_req_stock_out'] = 0

needQuantityReqStockOut.head(5)

,product_id,quantity_req_stockout,quantity_exported_req_stockout,need_quantity_req_stock_out
0,416,0.5,0.0,0.5
1,2448,3000.0,3000.0,0.0
2,3983,700.0,0.0,700.0
3,3989,2500.0,0.0,2500.0
4,6587,354.0,0.0,354.0


#### <input type="checkbox" disabled checked > 1.3 Cần - XKTT + YC xuất bù LSX `needQuantityReqMatOffset`


#### <input type="checkbox" disabled checked > 1.3.1 Cần - XKTT + YC xuất bù DM `needQuantityReqMatOffset`

In [180]:
dfRequestMaterialOffset = pd.read_csv('../data_csv/RequestMaterialOffset.csv')
dfRequestMaterialOffset

,product_id,material_offset_id,order_id,total_quota,req_mat_offset_status
0,8162,1,1,37.000,approval
1,5284,2,93,255.000,approval
2,8347,3,30,569.178,approval
3,8347,3,31,50.058,approval
4,8410,3,34,154.500,approval
...,...,...,...,...,...
2865,7814,1061,2259,105.000,approval
2866,5594,1061,2259,105.000,approval
2867,11353,1062,2261,15.000,approval
2868,11353,1063,2260,31.860,approval


In [181]:
dfRequestMaterialOffset[dfRequestMaterialOffset['product_id'] == 10667]

,product_id,material_offset_id,order_id,total_quota,req_mat_offset_status


#### <input type="checkbox" disabled checked > 1.3.2 Cần - XKTT + YC xuất bù LSX `needQuantityReqMatOffset`


In [182]:
dfReqMaterialOffsetProduction = pd.read_csv('../data_csv/ReqMaterialOffsetProduction.csv')
dfReqMaterialOffsetProduction

,code,request_id,production_order_id,status
0,30052022-1,1,29,approval
1,30052022-2,2,15,approval


In [183]:
dfReqMaterialOffsetProduction = pd.read_csv('../data_csv/ReqMaterialOffsetProduction.csv')
dfReqMaterialOffsetProductionStockOut = pd.read_csv('../data_csv/ReqMaterialOffsetProductionStockOut.csv')
dfReqMaterialOffsetProductionStockOut = dfReqMaterialOffsetProductionStockOut.replace(np.NaN , 0)
# dfReqMaterialOffsetProductionStockOut

dfReqMaterialOffsetProductionStockOut[dfReqMaterialOffsetProductionStockOut['product_id'] == 10667]

,product_id,order_id,request_id,production_order_id,is_exported,quantity_req_material_offset_production,assign_product_stockout_id,quantity_exported_req_material_offset_production


In [184]:
dfReqStockoutProductStockOutMerge           = dfReqMaterialOffsetProductionStockOut.merge(dfReqMaterialOffsetProduction, on = ['request_id', 'production_order_id'] , how = 'inner')
dfReqStockoutProductStockOutMergeNotComplete       = dfReqStockoutProductStockOutMerge[dfReqStockoutProductStockOutMerge['status'] != 'approval']
temp = dfReqStockoutProductStockOutMergeNotComplete.groupby(['product_id', 'request_id', 'production_order_id', 'quantity_req_material_offset_production'], as_index=False)['quantity_exported_req_material_offset_production'].sum()
needQuantityReqMatOffset = temp.groupby(['product_id'], as_index=False)['quantity_req_material_offset_production' , 'quantity_exported_req_material_offset_production'].sum()

needQuantityReqMatOffset

/tmp/ipykernel_8714/685297788.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  needQuantityReqMatOffset = temp.groupby(['product_id'], as_index=False)['quantity_req_material_offset_production' , 'quantity_exported_req_material_offset_production'].sum()


,product_id,quantity_req_material_offset_production,quantity_exported_req_material_offset_production


In [185]:
needQuantityReqMatOffset['need_quantity_req_mat_offset'] = needQuantityReqMatOffset['quantity_req_material_offset_production'] - needQuantityReqMatOffset['quantity_exported_req_material_offset_production'] 
needQuantityReqMatOffset

,product_id,quantity_req_material_offset_production,quantity_exported_req_material_offset_production,need_quantity_req_mat_offset


#### <input type="checkbox" disabled checked > 1.4 Number Need + Xuất đổi trả(chỉ tính sơ cấp) `needQuantityReturnProd`

In [186]:
dfProduct = pd.read_csv("../data_csv/Product.csv")
dfPrimary = dfProduct[dfProduct['is_outsourcing'] == 0]

In [187]:
dfReturnProduct = pd.read_csv('../data_csv/ReturnProduct.csv')

In [188]:
dfReturnProductStockOut = pd.read_csv('../data_csv/ReturnProductStockOut.csv')
dfReturnProductStockOut = dfReturnProductStockOut.replace(np.NaN , 0)
dfReturnProductStockOut[dfReturnProductStockOut['product_id'] == 10667]

,product_id,product_unit_id,return_product_id,quantity_return_product,purchase_order_id,outsourcing_order_id,is_complete_return_detail,assign_product_stockout_id,quantity_exported_return_product


In [189]:
dfReturnProductStockOut = pd.read_csv('../data_csv/ReturnProductStockOut.csv')
dfReturnProductStockOut = dfReturnProductStockOut.replace(np.NaN , 0)
temp = dfReturnProductStockOut.groupby(['product_id', 'product_unit_id', 'return_product_id', 'quantity_return_product', 
                        'purchase_order_id' , 'outsourcing_order_id', 'is_complete_return_detail'], as_index=False)['quantity_exported_return_product'].sum()

dfReturnProductStockOutSum = temp.groupby(['product_id', 'product_unit_id', 'return_product_id', 
                        'purchase_order_id' , 'outsourcing_order_id', 'is_complete_return_detail'], as_index=False)['quantity_return_product','quantity_exported_return_product'].sum()
dfReturnProductStockOutSum[dfReturnProductStockOutSum['product_id'] == 10667]

/tmp/ipykernel_8714/365735694.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfReturnProductStockOutSum = temp.groupby(['product_id', 'product_unit_id', 'return_product_id',


,product_id,product_unit_id,return_product_id,purchase_order_id,outsourcing_order_id,is_complete_return_detail,quantity_return_product,quantity_exported_return_product


In [190]:
dfReturnProductStockOut = pd.read_csv('../data_csv/ReturnProductStockOut.csv')
dfReturnProductStockOut = dfReturnProductStockOut.replace(np.NaN , 0)
temp = dfReturnProductStockOut.groupby(['product_id', 'product_unit_id', 'return_product_id', 'quantity_return_product', 
                        'purchase_order_id' , 'outsourcing_order_id', 'is_complete_return_detail'], as_index=False)['quantity_exported_return_product'].sum()

dfReturnProductStockOutSum = temp.groupby(['product_id', 'product_unit_id', 'return_product_id', 
                        'purchase_order_id' , 'outsourcing_order_id', 'is_complete_return_detail'], as_index=False)['quantity_return_product','quantity_exported_return_product'].sum()

dfReturnProductStockOutSumMerge = dfReturnProductStockOutSum.merge(dfReturnProduct, on = ['product_id', 'product_unit_id' ,'return_product_id'] , how = 'inner')
dfReturnProductStockOutSumMergeNotComplete  = dfReturnProductStockOutSumMerge[(dfReturnProductStockOutSumMerge['is_complete_return'] != 1) & (dfReturnProductStockOutSumMerge['is_complete_return_detail'] != 1)]


/tmp/ipykernel_8714/2545865031.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfReturnProductStockOutSum = temp.groupby(['product_id', 'product_unit_id', 'return_product_id',


In [191]:
needQuantityReturnProd = dfReturnProductStockOutSumMergeNotComplete[dfReturnProductStockOutSumMergeNotComplete['is_outsourcing'] == 0]

In [192]:
needQuantityReturnProd['need_quantity_return_prod'] = needQuantityReturnProd['quantity_return_product'] - needQuantityReturnProd['quantity_exported_return_product'] 
needQuantityReturnProd = needQuantityReturnProd.reset_index(drop = True)
needQuantityReturnProd

,product_id,product_unit_id,return_product_id,purchase_order_id,outsourcing_order_id,is_complete_return_detail,quantity_return_product,quantity_exported_return_product,code,qrcode,is_outsourcing,is_exported,is_complete_return,need_quantity_return_prod
0,96,9907,2,259,0.0,0,1082.9,990.20,29062022-2,1656466715-XKDT-2,0,0,0,92.70
1,790,12752,27,691,0.0,0,218.7,218.70,13102022-27,1665624060-XKDT-27,0,0,0,0.00
2,3728,11468,20,88,0.0,0,4317.5,4317.50,12092022-20,1662951998-XKDT-20,0,0,0,0.00
3,3729,11469,20,88,0.0,0,984.0,984.00,12092022-20,1662951998-XKDT-20,0,0,0,0.00
4,3731,11470,20,88,0.0,0,7942.0,7942.00,12092022-20,1662951998-XKDT-20,0,0,0,0.00
5,3796,3796,12,88,0.0,0,1000.0,490.78,15082022-12,1660545467-XKDT-12,0,0,0,509.22
6,3796,3796,18,687,0.0,0,500.0,457.20,29082022-18,1661737763-XKDT-18,0,0,0,42.80
7,3796,3796,19,884,0.0,0,1500.0,1371.60,08092022-19,1662629663-XKDT-19,0,0,0,128.40
8,3802,11475,6,88,0.0,0,25000.0,22850.00,27072022-6,1658905177-XKDT-6,0,0,0,2150.00
9,3825,11478,6,88,0.0,0,8000.0,7315.20,27072022-6,1658905177-XKDT-6,0,0,0,684.80


#### 1.5 Tổng số cần - XKTT `needQuantity`

In [193]:
needQuantityReturnProd

,product_id,product_unit_id,return_product_id,purchase_order_id,outsourcing_order_id,is_complete_return_detail,quantity_return_product,quantity_exported_return_product,code,qrcode,is_outsourcing,is_exported,is_complete_return,need_quantity_return_prod
0,96,9907,2,259,0.0,0,1082.9,990.20,29062022-2,1656466715-XKDT-2,0,0,0,92.70
1,790,12752,27,691,0.0,0,218.7,218.70,13102022-27,1665624060-XKDT-27,0,0,0,0.00
2,3728,11468,20,88,0.0,0,4317.5,4317.50,12092022-20,1662951998-XKDT-20,0,0,0,0.00
3,3729,11469,20,88,0.0,0,984.0,984.00,12092022-20,1662951998-XKDT-20,0,0,0,0.00
4,3731,11470,20,88,0.0,0,7942.0,7942.00,12092022-20,1662951998-XKDT-20,0,0,0,0.00
5,3796,3796,12,88,0.0,0,1000.0,490.78,15082022-12,1660545467-XKDT-12,0,0,0,509.22
6,3796,3796,18,687,0.0,0,500.0,457.20,29082022-18,1661737763-XKDT-18,0,0,0,42.80
7,3796,3796,19,884,0.0,0,1500.0,1371.60,08092022-19,1662629663-XKDT-19,0,0,0,128.40
8,3802,11475,6,88,0.0,0,25000.0,22850.00,27072022-6,1658905177-XKDT-6,0,0,0,2150.00
9,3825,11478,6,88,0.0,0,8000.0,7315.20,27072022-6,1658905177-XKDT-6,0,0,0,684.80


In [194]:
needQuantityTotal = needQuantityOrder.merge(needQuantityReqStockOut, on = 'product_id', how = 'outer').merge(needQuantityReqMatOffset, 
                                                on = 'product_id', how = 'outer').merge(needQuantityReturnProd, on = 'product_id', how = 'outer')
needQuantityTotal = needQuantityTotal.replace(np.NaN, 0)
needQuantityTotal['need_quantity'] = needQuantityTotal['need_quantity_order'] + needQuantityTotal[
            'need_quantity_req_stock_out'] + needQuantityTotal['need_quantity_req_mat_offset'] + needQuantityTotal['need_quantity_return_prod']
needQuantityTotal[needQuantityTotal['product_id'] == 3447 ]

,product_id,total_quota,quantity_production_order,quantity_exported_production_order,need_quantity_order,quantity_req_stockout,quantity_exported_req_stockout,need_quantity_req_stock_out,quantity_req_material_offset_production,quantity_exported_req_material_offset_production,need_quantity_req_mat_offset,product_unit_id,return_product_id,purchase_order_id,outsourcing_order_id,is_complete_return_detail,quantity_return_product,quantity_exported_return_product,code,qrcode,is_outsourcing,is_exported,is_complete_return,need_quantity_return_prod,need_quantity
537,3447,15.18,15.12,15.12,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.06


In [195]:
needQuantityTotal = needQuantityTotal.sort_values(by=['product_id']).reset_index(drop = True)
needQuantity = needQuantityTotal[['product_id','need_quantity']]
needQuantity

,product_id,need_quantity
0,1,118.41
1,2,4384.46
2,6,51.00
3,7,60.46
4,8,69.38
...,...,...
2114,11452,208.00
2115,11453,22.03
2116,11456,655.00
2117,11457,2.24


In [196]:
dfMaterialReports

,product_id,created_at,updated_at,deleted_at,product_unit_id,need_quantity,ordered_quantity,need_order_quantity,in_stock,need_for_outsourcing,is_calculated,outsourcing_stock_out,temporary_quantity
0,1,2022-06-27 11:20:23+00:00,2022-12-06 08:50:09+00:00,NaN,16620,118.41,0.0,-3855.425990,6301.91,2328.074010,1,0.00,3855.425990
1,2,2022-06-26 01:24:29+00:00,2022-12-06 09:42:47+00:00,NaN,10127,4299.46,0.0,6221.020621,13742.73,15664.290621,1,0.00,-6221.020621
2,3,2022-06-26 06:31:44+00:00,2022-12-06 05:05:05+00:00,NaN,8583,0.00,0.0,-4365.100000,4437.70,72.600000,1,0.00,4365.100000
3,4,2022-06-26 01:19:28+00:00,2022-12-06 05:12:18+00:00,NaN,9913,0.00,0.0,-2638.978000,3270.90,631.922000,1,0.00,2638.978000
4,5,2022-06-26 01:19:00+00:00,2022-12-06 07:54:09+00:00,NaN,9912,0.00,0.0,-1024.100700,2324.08,1655.819300,1,355.84,1024.100700
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11329,11468,2022-12-05 14:12:16+00:00,2022-12-06 03:10:01+00:00,NaN,21746,0.00,2.0,-2.000000,0.00,0.000000,1,0.00,2.000000
11330,11469,2022-12-05 14:12:17+00:00,2022-12-06 03:10:02+00:00,NaN,21747,0.00,2.0,-2.000000,0.00,0.000000,1,0.00,2.000000
11331,11470,2022-12-05 15:43:38+00:00,2022-12-06 06:15:48+00:00,NaN,21748,0.00,0.0,-6.000000,6.00,0.000000,1,0.00,6.000000
11332,11471,2022-12-05 15:43:38+00:00,2022-12-06 06:15:49+00:00,NaN,21749,0.00,0.0,-1.000000,1.00,0.000000,1,0.00,1.000000


In [197]:
needQuantityRp = dfMaterialReports[['product_id','need_quantity']]
compare = needQuantity.merge(needQuantityRp , on = 'product_id' , how = 'inner')
compare['quantity_compare'] = round((compare['need_quantity_y'] - compare['need_quantity_x']) , 4 ) 
compare.loc[compare['quantity_compare'] < 0]

,product_id,need_quantity_x,need_quantity_y,quantity_compare
1,2,4384.46,4299.46,-85.0
102,609,354.84,325.84,-29.0
1379,8508,43495.07,42929.07,-566.0
1852,10667,33.02,10.02,-23.0


In [198]:
needQuantity[needQuantity['product_id'] == 86]

,product_id,need_quantity
19,86,1.24


In [199]:
dfProduct[dfProduct['product_id'] == 86]

,product_id,is_outsourcing,product_code,quantity_product
78,86,1,1-CVAC06-BLK-V1-HV,6.39


### 2. Số cần của thứ cấp
- Đơn hàng bán của TC
- Phiếu yêu "cầu xuất kho của TC
- YC xuất bù LSX

#### Xây dựng Multi-Level BOM

In [16]:
dfOrder = pd.read_csv("../data_csv/Order.csv")
dfProduct = pd.read_csv("../data_csv/Product.csv")
dfRelationship = pd.read_csv("../data_csv/Relationship.csv")

In [17]:
def mergeMaterialOrder(dfRelationship, dfProduct, dfOrder):
   dfOrder = dfOrder.rename(columns = {"product_id":"parent_product"})
   dfRelationship = dfRelationship.rename(columns = {"product_id":"child_product" , 'outsourcing_product_id' : 'parent_product'})
   colProduct = ['product_id', 'product_code' , 'quantity_product' , 'is_outsourcing']
   dfProduct = dfProduct.reindex(columns=colProduct).rename(columns = {"product_id":"parent_product"})

   dfRelationship_dropdup = dfRelationship.drop_duplicates(keep='first').reset_index(drop = True).sort_values(by=['parent_product']).reset_index(drop = True)
   dfRelationship_merge = pd.merge(dfRelationship_dropdup , dfProduct[['parent_product','quantity_product']], left_on='child_product', right_on='parent_product', how='inner').drop(columns = ['parent_product_y']).rename(columns = {"parent_product_x":"parent_product"})
   dfRelationship_merge = dfRelationship_merge.reset_index(drop = True)
   g = dfRelationship_merge.groupby(["parent_product"]).cumcount().add(1)
   dfRelationshipMerge = dfRelationship_merge.set_index(["parent_product", g]).unstack(fill_value=0).sort_index(axis=1, level=1)
   dfRelationshipMerge.columns = ["{}{}".format(a, b) for a, b in dfRelationshipMerge.columns]

   dfRelationshipMerge = dfRelationshipMerge.reset_index()

   dfRelationshipMerge = dfRelationshipMerge.replace(np.nan, 0)
   dfTotalMate = pd.merge(dfProduct, dfRelationshipMerge, on='parent_product', how='left')
   dfTotalMate = dfTotalMate.replace(np.nan, 0)

   dfTotalMate = dfTotalMate.drop(dfTotalMate[(dfTotalMate['child_product1'] == 0) & (dfTotalMate['is_outsourcing'] == 1)].index).reset_index(drop = True)

   dfTotalMate = dfTotalMate.sort_values(by= ['parent_product']).reset_index(drop = True) 
   dfTotal = pd.merge(dfOrder , dfTotalMate, on=['product_code', 'parent_product'], how='left')

   # dfTotal = dfTotal[:1000] 
   dfTotalReverse = dfTotal[::-1]
   dfTotalReverse = dfTotalReverse.reset_index(drop = True)
   dfTotalReverse = dfTotalReverse.drop(dfTotalReverse[np.isnan(dfTotalReverse['parent_product']) == True].index).reset_index(drop = True)
   dfTotalReverse
   return dfTotalMate, dfTotalReverse

In [18]:
dfSecondary = dfProduct.loc[dfProduct['is_outsourcing'] == 1].reset_index(drop = True)
dfPrimary = dfProduct.loc[dfProduct['is_outsourcing'] == 0].reset_index(drop = True)

In [19]:
import time

In [20]:
dfTotalMate, dfTotalReverse = mergeMaterialOrder(dfRelationship , dfProduct, dfOrder)
dfSecondaryRelation = dfTotalReverse.loc[dfTotalReverse['is_outsourcing'] == 1].reset_index(drop = True)

In [21]:
dict = {"parent_product": [] ,"child_product" : [],"quota_outsourcing" : [] , "level" : [] , 'size_level_next': []}

def appendDict(dict, parent_product , child_product, quota_outsourcing, level, size_level_next):
    dict["parent_product"].append(parent_product)
    dict["child_product"].append(child_product)
    dict["quota_outsourcing"].append(quota_outsourcing)
    dict["level"].append(level)
    dict["size_level_next"].append(size_level_next)
    

In [22]:
# calculate the quantity of primary materials of secondary materials
def cal_size_level_next(aMate):
    size_level = 0
    for i in range(1, 6):
        if (aMate['child_product{}'.format(i)] != 0):
            size_level = size_level + 1
    return size_level

In [23]:
# recursive calculator number need with child is primary, not secondary
def recursivelyNumberNeed(dict, dfTotalMate, parent_product ,product_id, quota, level, size_level_next):
    aMate = dfTotalMate.loc[np.where(dfTotalMate['parent_product'] == product_id)[0][0]] 
    if ((aMate['is_outsourcing'] == 0) & (size_level_next == 0)): 
        appendDict(dict, parent_product , aMate['parent_product'], quota, level, 0) 
    elif ((aMate['is_outsourcing'] == 0) & (size_level_next != 0)): 
        appendDict(dict, parent_product , aMate['parent_product'], quota, level, 0)
        # if(size_level_next >= 2):
        #     level = level - 1
    elif((aMate['is_outsourcing'] != 0) & (size_level_next != 0)): 
        size_level_next = cal_size_level_next(aMate) 
        for i in range(1, size_level_next + 1):
            dict["quota_outsourcing"].append(quota)
            level = level - (i - 1)
            dict["level"].append(level)
            dict["size_level_next"].append(size_level_next)
            quota = quota * aMate['quota_outsourcing{}'.format(i)]
            level = level + 1
            dict["parent_product"].append(parent_product)
            child_product = aMate['child_product{}'.format(i)]
            dict["child_product"].append(product_id)
            recursivelyNumberNeed(dict, dfTotalMate, parent_product, child_product, quota, level, size_level_next)

In [24]:
aMaterial = dfTotalMate[dfTotalMate['parent_product'] == 7245]
aMaterial

,parent_product,product_code,quantity_product,is_outsourcing,child_product1,product_code1,quantity_product1,quota_outsourcing1,child_product2,product_code2,quantity_product2,quota_outsourcing2,child_product3,product_code3,quantity_product3,quota_outsourcing3,child_product4,product_code4,quantity_product4,quota_outsourcing4,child_product5,product_code5,quantity_product5,quota_outsourcing5
7053,7245,4-RPHW1R-BRO-N1-36,0.0,1,1681.0,1-RPHW1R-BRO-L1-HF,0.0,0.031,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0


In [25]:
# appendDict(dict, material_order , child_product, quota_outsourcing, level, size_level_next):
# recursive calculator secondary need
def recursivelyCalculateSecondaryNeed(dfTotalMate):
    for i in range(0 , len(dfTotalMate)):
        aMaterial = dfTotalMate.loc[i]
        parent_product = aMaterial['parent_product']
        size_level_next = cal_size_level_next(aMaterial)
        if (size_level_next == 0): 
            level = 1
            appendDict(dict, parent_product , 0, 0, level, 0)
        else:
            for i in range(1 , size_level_next + 1):
                quota = 1
                level = 2
                quota = quota * aMaterial['quota_outsourcing{}'.format(i)]
                product_id = aMaterial['child_product{}'.format(i)]
                # appendDict(dict, parent_product , child_product, quota, level, size_level_next) 
                recursivelyNumberNeed(dict, dfTotalMate, parent_product, product_id, quota, level, size_level_next)
                        
    return dict
            
dict = recursivelyCalculateSecondaryNeed(dfTotalMate)
multiLevelBOMOrder = pd.DataFrame.from_dict(dict)

In [26]:
dfTotalReverse

,parent_product,order_id,product_code,order_status,position,total_quota,complete_order,booking_status,quantity_product,is_outsourcing,child_product1,product_code1,quantity_product1,quota_outsourcing1,child_product2,product_code2,quantity_product2,quota_outsourcing2,child_product3,product_code3,quantity_product3,quota_outsourcing3,child_product4,product_code4,quantity_product4,quota_outsourcing4,child_product5,product_code5,quantity_product5,quota_outsourcing5
0,9476,2518,9-TL5340-Y00-5L-BN,approval,Tấm lót,2.0000,0,review,0.0000,0.0,0.0,0,0.00,0.0000,0.0,0,0.0000,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
1,8464,2518,9-554331-Y00-5L-BN,approval,Thùng lớn,2.0000,0,review,12.0000,0.0,0.0,0,0.00,0.0000,0.0,0,0.0000,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
2,6731,2518,9-B03680-WHI-00-HN,approval,Bao nylon,6.0000,0,review,63.9355,0.0,0.0,0,0.00,0.0000,0.0,0,0.0000,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
3,8916,2518,9-GL2608-450-00-TA,approval,Giấy lót nón,150.0000,0,review,0.0000,1.0,6608.0,9-GR450G-000-00-TF,480.00,0.0278,0.0,0,0.0000,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
4,5948,2518,7-CM0BLK-BLK-50-PP,approval,chỉ may black,12.0000,0,review,201.2009,0.0,0.0,0,0.00,0.0000,0.0,0,0.0000,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55960,74,1,1-CVAC04-SCA-00-HV,completed,Đuôi phải,7.8280,1,approval,1131.9040,0.0,0.0,0,0.00,0.0000,0.0,0,0.0000,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
55961,122,1,1-CVAC12-ROL-00-IW,completed,Hông trái,6.4000,1,approval,91.1371,0.0,0.0,0,0.00,0.0000,0.0,0,0.0000,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
55962,74,1,1-CVAC04-SCA-00-HV,completed,Hông phải+nút,6.7200,1,approval,1131.9040,0.0,0.0,0,0.00,0.0000,0.0,0,0.0000,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
55963,124,1,1-CVAC12-ROL-L2-IW,completed,Mặt trái,6.5625,1,approval,1.4375,1.0,1.0,1-LUOIMA-BLK-80-CM,6301.91,1.0000,122.0,1-CVAC12-ROL-00-IW,91.1371,1.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0


In [33]:
multiLevelBOMOrder[multiLevelBOMOrder['level'] == 5]

,parent_product,child_product,quota_outsourcing,level,size_level_next
9609,8410,1050.0,0.0690,5,0
10697,8909,1050.0,0.0640,5,0
13841,10687,9103.0,0.0672,5,0
14848,11320,9103.0,0.0672,5,0


In [34]:
multiLevelBOMOrder[multiLevelBOMOrder['parent_product'] == 8410]

,parent_product,child_product,quota_outsourcing,level,size_level_next
9600,8410,8312.0,1.000,2,1
9601,8410,8311.0,0.026,3,0
9602,8410,8314.0,1.000,2,1
9603,8410,6643.0,0.026,3,0
9604,8410,8408.0,1.000,2,1
9605,8410,1066.0,0.069,3,2
9606,8410,2.0,0.069,4,0
9607,8410,1066.0,0.069,3,2
9608,8410,1061.0,0.069,4,1
9609,8410,1050.0,0.069,5,0


#### 2.1 Đơn hàng bán của TC

In [914]:
multiLevelBOMOrder

,parent_product,child_product,quota_outsourcing,level,size_level_next
0,1,0.0,0.0,1,0
1,2,0.0,0.0,1,0
2,3,0.0,0.0,1,0
3,4,0.0,0.0,1,0
4,5,0.0,0.0,1,0
...,...,...,...,...,...
15088,11468,777.0,1.0,2,0
15089,11469,777.0,1.0,2,0
15090,11470,0.0,0.0,1,0
15091,11471,0.0,0.0,1,0


In [915]:
multiLevelBOMOrder[multiLevelBOMOrder['parent_product'] == 7480]

,parent_product,child_product,quota_outsourcing,level,size_level_next
8147,7480,8312.0,1.0000,2,1
8148,7480,8311.0,0.0260,3,0
8149,7480,11364.0,1.0000,2,1
8150,7480,398.0,0.0672,3,2
8151,7480,1.0,0.0672,4,0
8152,7480,398.0,0.0672,3,2
8153,7480,395.0,0.0672,4,0


In [916]:
dfOrderRelationship = multiLevelBOMOrder.merge(dfTotalReverse[['parent_product', 'order_id' , 'position' , 'total_quota' , 'quantity_product']], on =  'parent_product' , how = 'outer').sort_values(by = 'order_id' , axis = 0 , ascending=False).reset_index(drop = True)
dfOrderRelationship

,parent_product,child_product,quota_outsourcing,level,size_level_next,order_id,position,total_quota,quantity_product
0,3983,7824.0,0.0080,2.0,0.0,2518.0,Dây size,99.00,0.000000
1,5238,0.0,0.0000,1.0,0.0,2518.0,Nhãn dệt đai,152.00,643.000000
2,2213,0.0,0.0000,1.0,0.0,2518.0,Két trên+nút,3.63,223.242200
3,8508,793.0,0.0241,2.0,0.0,2518.0,Viền,222.75,2456.600001
4,6580,0.0,0.0000,1.0,0.0,2518.0,Nút sắt,155.00,142912.720000
...,...,...,...,...,...,...,...,...,...
81193,11468,777.0,1.0000,2.0,0.0,NaN,NaN,NaN,NaN
81194,11469,777.0,1.0000,2.0,0.0,NaN,NaN,NaN,NaN
81195,11470,0.0,0.0000,1.0,0.0,NaN,NaN,NaN,NaN
81196,11471,0.0,0.0000,1.0,0.0,NaN,NaN,NaN,NaN


In [917]:
dfOrderRelationship[dfOrderRelationship['parent_product'] == 8916]

,parent_product,child_product,quota_outsourcing,level,size_level_next,order_id,position,total_quota,quantity_product
19,8916,6608.0,0.0278,2.0,0.0,2518.0,Giấy lót nón,150.0,0.0
73,8916,6608.0,0.0278,2.0,0.0,2516.0,Giấy lót nón,151.0,0.0
118,8916,6608.0,0.0278,2.0,0.0,2514.0,Giấy lót nón,7000.0,0.0
158,8916,6608.0,0.0278,2.0,0.0,2513.0,Giấy lót nón,1000.0,0.0
548,8916,6608.0,0.0278,2.0,0.0,2501.0,Giấy lót nón,201.0,0.0
...,...,...,...,...,...,...,...,...,...
59084,8916,6608.0,0.0278,2.0,0.0,537.0,Giấy lót nón,145.0,0.0
59088,8916,6608.0,0.0278,2.0,0.0,536.0,Giấy lót nón,51.0,0.0
59115,8916,6608.0,0.0278,2.0,0.0,535.0,Giấy lót nón,101.0,0.0
59233,8916,6608.0,0.0278,2.0,0.0,532.0,Giấy lót nón,2501.0,0.0


In [918]:
dfOrderRelationship[dfOrderRelationship['parent_product'] == 8347].head(15)

,parent_product,child_product,quota_outsourcing,level,size_level_next,order_id,position,total_quota,quantity_product
5,8347,8313.0,1.0000,2.0,1.0,2518.0,Đai,99.00,2854.662
6,8347,7923.0,0.0260,3.0,0.0,2518.0,Đai,99.00,2854.662
9,8347,82.0,0.0672,3.0,0.0,2518.0,Đai,99.00,2854.662
13,8347,8312.0,1.0000,2.0,1.0,2518.0,Đai,99.00,2854.662
14,8347,8326.0,1.0000,2.0,1.0,2518.0,Đai,99.00,2854.662
22,8347,8311.0,0.0260,3.0,0.0,2518.0,Đai,99.00,2854.662
136,8347,82.0,0.0672,3.0,0.0,2513.0,Đai,618.00,2854.662
142,8347,7923.0,0.0260,3.0,0.0,2513.0,Đai,618.00,2854.662
144,8347,8313.0,1.0000,2.0,1.0,2513.0,Đai,618.00,2854.662
148,8347,8312.0,1.0000,2.0,1.0,2513.0,Đai,618.00,2854.662


In [919]:
dfTotalMate[dfTotalMate['parent_product'] == 8431]
# dfSecondaryRelation[dfSecondaryRelation['order_id'] == 47]

,parent_product,product_code,quantity_product,is_outsourcing,child_product1,product_code1,quantity_product1,quota_outsourcing1,child_product2,product_code2,quantity_product2,quota_outsourcing2,child_product3,product_code3,quantity_product3,quota_outsourcing3,child_product4,product_code4,quantity_product4,quota_outsourcing4,child_product5,product_code5,quantity_product5,quota_outsourcing5
8148,8431,4-CVAC08-WHI-I5-30,107.4,1,8372.0,9-GIA80G-WHI-35-TA,0.0,1.0,8427.0,9-GOLDAI-WHI-35-TA,0.0,1.0,8430.0,9-CVAC08-WHI-00-55,0.0,1.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0


##### Relationship between of Order detail and Product detail

In [920]:
# dfTotalMate, dfTotalReverse = mergeMaterialOrder(dfRelationship , dfProduct, dfOrder)

In [921]:
# dict = {'order_id': [],"material_order": [] ,"child_product" : [] , "total_quota": [],"quota_outsourcing" : [] , "level" : [] , 'size_level_next': []}

# def appendDict(dict, order_id, material_order , child_product, total_quota, quota_outsourcing, level, size_level_next):
#     dict["order_id"].append(order_id)
#     dict["material_order"].append(material_order)
#     dict["child_product"].append(child_product)
#     dict["total_quota"].append(total_quota)
#     dict["quota_outsourcing"].append(quota_outsourcing)
#     dict["level"].append(level)
#     dict["size_level_next"].append(size_level_next)
    

In [922]:
# calculate the quantity of primary materials of secondary materials
# def cal_size_level_next(aMate):
#     size_level = 0
#     for i in range(1, 6):
#         if (aMate['child_product{}'.format(i)] != 0):
#             size_level = size_level + 1
#     return size_level

In [923]:
# aMaterial = dfTotalMate[dfTotalMate['parent_product'] == 11463]
# aMaterial

In [924]:
# aMaterial = dfTotalReverse[dfTotalReverse['parent_product'] == 11463]
# aMaterial

In [925]:
# # recursive calculator number need with child is primary, not secondary
# def recursivelyNumberNeed(dict, product_id, dfTotalMate, quota, material_order, order_id, total_quota, level, size_level_next):
#     aMate = dfTotalMate.loc[np.where(dfTotalMate['parent_product'] == product_id)[0][0]] 
#     if ((aMate['is_outsourcing'] == 0) & (size_level_next == 0)): 
#         appendDict(dict, order_id, material_order , aMate['parent_product'], total_quota, quota, level, 0) 
#     elif ((aMate['is_outsourcing'] == 0) & (size_level_next != 0)): 
#         appendDict(dict, order_id, material_order , aMate['parent_product'], total_quota, quota, level, 0)
#     elif((aMate['is_outsourcing'] != 0) & (size_level_next != 0)): 
#         size_level_next = cal_size_level_next(aMate) 
#         for i in range(1, size_level_next + 1):
#             dict["quota_outsourcing"].append(quota)
#             level = level - (i - 1)
#             dict["level"].append(level)
#             dict["size_level_next"].append(size_level_next)
#             quota = quota * aMate['quota_outsourcing{}'.format(i)]
#             level = level + 1
#             # dict["child_product"].append(product_id)
#             child_product = aMate['child_product{}'.format(i)]
#             dict["order_id"].append(order_id)
#             dict["material_order"].append(material_order)
#             dict["total_quota"].append(total_quota)
#             dict["child_product"].append(product_id)
#             recursivelyNumberNeed(dict, child_product, dfTotalMate, quota, material_order, order_id, total_quota, level, size_level_next)


In [926]:
# # recursive calculator secondary need
# def recursivelyCalculateSecondaryNeed(dfSecondaryRelation, dfTotalMate):
#     for i in range(0 , len(dfSecondaryRelation)):
#         aMaterial = dfSecondaryRelation.loc[i]
#         order_id = aMaterial['order_id']
#         total_quota = aMaterial['total_quota']
#         material_order = aMaterial['parent_product']
        
#         size_level_next = cal_size_level_next(aMaterial)
#         if (size_level_next == 0): 
#             appendDict(dict, order_id, material_order , order_id, total_quota, quota, level, 0)
#         else:
#             for i in range(1 , size_level_next + 1):
#                 quota = 1
#                 level = 2
                
#                 quota = quota * aMaterial['quota_outsourcing{}'.format(i)]
#                 product_id = aMaterial['child_product{}'.format(i)]
#                 # appendDict(dict, order_id, material_order , product_id , total_quota, quota, level, 0)
#                 recursivelyNumberNeed(dict, product_id, dfTotalMate, quota, material_order, order_id, total_quota, level, size_level_next)
        
#     return dict
            
# dict = recursivelyCalculateSecondaryNeed(dfSecondaryRelation, dfTotalMate)
# multiLevelBOMOrder = pd.DataFrame.from_dict(dict)


In [927]:
# dfTotalMate[dfTotalMate['parent_product'] == 11463]

#### Tính cần của thứ cấp bởi sơ cấp `needForOutsourcingOrder`

In [928]:
multiLevelBOMOrderDropDup = multiLevelBOMOrder.drop_duplicates(subset=['parent_product' , 'child_product' , 'quota_outsourcing' , 'level' , 'size_level_next'], keep='first').reset_index(drop= True)
multiLevelBOMOrderDropDup

,parent_product,child_product,quota_outsourcing,level,size_level_next
0,1,0.0,0.0,1,0
1,2,0.0,0.0,1,0
2,3,0.0,0.0,1,0
3,4,0.0,0.0,1,0
4,5,0.0,0.0,1,0
...,...,...,...,...,...
14696,11468,777.0,1.0,2,0
14697,11469,777.0,1.0,2,0
14698,11470,0.0,0.0,1,0
14699,11471,0.0,0.0,1,0


In [929]:
multiLevelBOMOrderDropDup[multiLevelBOMOrderDropDup['parent_product'] == 11413]

,parent_product,child_product,quota_outsourcing,level,size_level_next
14618,11413,11378.0,0.0276,2,2
14619,11413,5.0,0.0276,3,0
14620,11413,11355.0,0.0276,3,0


In [930]:
dfNeedSecondary = needQuantityOrder.merge(dfSecondary , on = 'product_id' , how = 'inner')
dfNeedSecondaryMerge = dfNeedSecondary.merge(multiLevelBOMOrderDropDup, left_on = 'product_id' , right_on = 'parent_product' , how = 'left')
dfNeedSecondaryMerge['need_for_outsourcing_order'] = dfNeedSecondaryMerge['need_quantity_order'] * dfNeedSecondaryMerge['quota_outsourcing']
dfNeedSecondaryMerge[dfNeedSecondaryMerge['child_product'] == 11378]

,product_id,total_quota,quantity_production_order,quantity_exported_production_order,need_quantity_order,is_outsourcing,product_code,quantity_product,parent_product,child_product,quota_outsourcing,level,size_level_next,need_for_outsourcing_order
1703,11413,407.0,0.0,0.0,407.0,1,4-P2585Q-CTG-L3-32,0.0,11413,11378.0,0.0276,2,2,11.2332
1706,11414,187.0,0.0,0.0,187.0,1,4-P2585Q-CTG-L3-28,0.0,11414,11378.0,0.0241,2,2,4.5067


In [931]:
dfNeedSecondaryMerge[dfNeedSecondaryMerge['child_product'] == 11378]

,product_id,total_quota,quantity_production_order,quantity_exported_production_order,need_quantity_order,is_outsourcing,product_code,quantity_product,parent_product,child_product,quota_outsourcing,level,size_level_next,need_for_outsourcing_order
1703,11413,407.0,0.0,0.0,407.0,1,4-P2585Q-CTG-L3-32,0.0,11413,11378.0,0.0276,2,2,11.2332
1706,11414,187.0,0.0,0.0,187.0,1,4-P2585Q-CTG-L3-28,0.0,11414,11378.0,0.0241,2,2,4.5067


In [932]:
needForOutsourcingOrder = dfNeedSecondaryMerge.groupby(['product_id','child_product'], as_index=False)['need_for_outsourcing_order'].sum()
needForOutsourcingOrder

,product_id,child_product,need_for_outsourcing_order
0,84,82.0,0.0000
1,86,82.0,1.2400
2,86,793.0,1.2400
3,88,82.0,2.0100
4,88,2350.0,2.0100
...,...,...,...
1734,11457,1294.0,2.2400
1735,11462,6643.0,0.1208
1736,11462,8372.0,4.0000
1737,11462,11118.0,0.1412


#### 2.2 Phiếu yêu cầu xuất kho của TC `needForOutsourcingReqStockOut`

In [933]:
needQuantityReqStockOut.head(5)

,product_id,quantity_req_stockout,quantity_exported_req_stockout,need_quantity_req_stock_out
0,416,0.5,0.0,0.5
1,2448,3000.0,3000.0,0.0
2,3983,700.0,0.0,700.0
3,3989,2500.0,0.0,2500.0
4,6587,354.0,0.0,354.0


In [934]:
needQuantityReqStockOut

,product_id,quantity_req_stockout,quantity_exported_req_stockout,need_quantity_req_stock_out
0,416,0.5,0.00,0.50
1,2448,3000.0,3000.00,0.00
2,3983,700.0,0.00,700.00
3,3989,2500.0,0.00,2500.00
4,6587,354.0,0.00,354.00
5,6590,153880.0,66263.50,87616.50
6,6594,4420.0,0.00,4420.00
7,6597,2745.0,0.00,2745.00
8,6598,552.0,0.00,552.00
9,6599,2745.0,0.00,2745.00


In [935]:
dfNeedSecondary = needQuantityReqStockOut.merge(dfSecondary , on = 'product_id' , how = 'inner')
dfNeedSecondaryMerge = dfNeedSecondary.merge(multiLevelBOMOrderDropDup, left_on = 'product_id' , right_on = 'parent_product' , how = 'left')
dfNeedSecondaryMerge['need_for_outsourcing_req_stock_out'] = dfNeedSecondaryMerge['need_quantity_req_stock_out'] * dfNeedSecondaryMerge['quota_outsourcing']
dfNeedSecondaryMerge.head(5)

,product_id,quantity_req_stockout,quantity_exported_req_stockout,need_quantity_req_stock_out,is_outsourcing,product_code,quantity_product,parent_product,child_product,quota_outsourcing,level,size_level_next,need_for_outsourcing_req_stock_out
0,3983,700.0,0.00,700.00,1,5-DAFELT-BLK-07-TA,0.000000,3983,7824.0,0.0080,2,0,5.600000
1,3989,2500.0,0.00,2500.00,1,5-DAFELT-WHI-07-TA,0.000000,3989,8278.0,0.0080,2,0,20.000000
2,6885,3000.0,364.52,2635.48,1,4-KATE00-WHI-N0-28,1630.105000,6885,904.0,0.0241,2,0,63.515068
3,8380,400.0,255.71,144.29,1,4-THUNME-BLK-DB-30,500.000001,8380,8313.0,1.0000,2,1,144.290000
4,8380,400.0,255.71,144.29,1,4-THUNME-BLK-DB-30,500.000001,8380,7923.0,0.0260,3,0,3.751540


In [936]:
needForOutsourcingReqStockOut = dfNeedSecondaryMerge.groupby(['child_product'], as_index=False)['need_for_outsourcing_req_stock_out'].sum()
needForOutsourcingReqStockOut.head(5)

,child_product,need_for_outsourcing_req_stock_out
0,2.0,33.710000
1,904.0,63.515068
2,2259.0,12.915702
3,2260.0,92.360000
4,6643.0,38.080000


#### 2.3 YC xuất bù LSX `needForOutsourcingReqOffset`

In [937]:
needQuantityReqMatOffset

,product_id,quantity_req_material_offset_production,quantity_exported_req_material_offset_production,need_quantity_req_mat_offset


In [938]:
dfNeedSecondary = needQuantityReqMatOffset.merge(dfSecondary , on = 'product_id' , how = 'inner')
dfNeedSecondaryMerge = dfNeedSecondary.merge(multiLevelBOMOrderDropDup, left_on = 'product_id' , right_on = 'parent_product' , how = 'left')
dfNeedSecondaryMerge['need_for_outsourcing_req_offset'] = dfNeedSecondaryMerge['need_quantity_req_mat_offset'] * dfNeedSecondaryMerge['quota_outsourcing']
dfNeedSecondaryMerge

,quantity_req_material_offset_production,quantity_exported_req_material_offset_production,need_quantity_req_mat_offset,product_id,is_outsourcing,product_code,quantity_product,parent_product,child_product,quota_outsourcing,level,size_level_next,need_for_outsourcing_req_offset


In [939]:
needForOutsourcingReqOffset = dfNeedSecondaryMerge.groupby(['child_product'], as_index=False)['need_for_outsourcing_req_offset'].sum()
needForOutsourcingReqOffset

,child_product,need_for_outsourcing_req_offset


#### 2.4 Số cần của thứ cấp `needForOutsourcing`

In [940]:
needForOutsourcing = needForOutsourcingOrder.merge(needForOutsourcingReqStockOut, on = 'child_product', how = 'outer').merge(needForOutsourcingReqOffset, 
                                                on = 'child_product', how = 'outer')
needForOutsourcing = needForOutsourcing.replace(np.NaN, 0).rename(columns= {'child_product' : 'product_id'})
needForOutsourcing['need_for_outsourcing'] = needForOutsourcing['need_for_outsourcing_order'] + needForOutsourcing['need_for_outsourcing_req_stock_out'] + needForOutsourcing['need_for_outsourcing_req_offset']
needForOutsourcing = needForOutsourcing[['product_id' , 'need_for_outsourcing']]
needForOutsourcing

,product_id,product_id,need_for_outsourcing
0,84,82.0,0.000000
1,86,82.0,1.240000
2,88,82.0,2.010000
3,91,82.0,-2.330000
4,7658,82.0,0.046428
...,...,...,...
1734,11444,11446.0,630.000000
1735,11447,177.0,3.010000
1736,11457,1294.0,2.240000
1737,11462,11118.0,0.141200


In [942]:
needForOutsourcingRp = dfMaterialReports[['product_id','need_for_outsourcing']]
needForOutsourcingRp

,product_id,need_for_outsourcing
0,1,2328.074010
1,2,15664.290621
2,3,72.600000
3,4,631.922000
4,5,1655.819300
...,...,...
11329,11468,0.000000
11330,11469,0.000000
11331,11470,0.000000
11332,11471,0.000000


In [941]:
needForOutsourcingRp = dfMaterialReports[['product_id','need_for_outsourcing']]
compare = needForOutsourcing.merge(needForOutsourcingRp , on = 'product_id' , how = 'inner')
compare['quantity_compare'] = round((compare['need_for_outsourcing_y'] - compare['need_for_outsourcing_x']) , 4 ) 
compare.loc[compare['quantity_compare'] < 0]

ValueError: The column label 'product_id' is not unique.

### 3 Total

In [840]:
# inStock
# orderedQuantity
# outSourcingStockOut
# needForOutsourcing
# needQuantity

NameError: name 'outSourcingStockOut' is not defined

In [841]:
orderedQuantity

,product_id,quantity_not_arrived,material_not_in_stock_outSourcing,material_not_in_stock_request,ordered_quantity
0,6,88.26,0.0,0.0,88.26
1,7,88.26,0.0,0.0,88.26
2,8,88.26,0.0,0.0,88.26
3,82,914.00,0.0,0.0,914.00
4,86,0.00,3.5,0.0,3.50
...,...,...,...,...,...
642,11453,0.00,22.0,0.0,22.00
643,11466,4.00,0.0,0.0,4.00
644,11468,0.00,2.0,0.0,2.00
645,11469,0.00,2.0,0.0,2.00


In [842]:
temporaryQuantity = inStock.merge(orderedQuantity[['product_id' , 'ordered_quantity']], on = 'product_id', how = 'outer').merge(outsourcingStockOut, 
                                    on = 'product_id', how = 'outer').merge(needForOutsourcing, 
                                    on = 'product_id', how = 'outer').merge(needQuantity, 
                                    on = 'product_id', how = 'outer')
                                    
temporaryQuantity = temporaryQuantity.replace(np.NaN , 0).reset_index(drop = True)
temporaryQuantity

,product_id,in_stock,ordered_quantity,convert_total_quota,quantity_exported_outsourcing,outsourcing_stock_out,need_for_outsourcing,need_quantity
0,1.0,6301.91,0.0,255.2,233.50,0.00,3287.660660,118.41
1,2.0,13742.73,0.0,41.5,6.50,0.00,22719.221511,4384.46
2,3.0,4437.70,0.0,0.0,0.00,0.00,72.600000,0.00
3,4.0,3270.90,0.0,0.0,0.00,0.00,680.682354,0.00
4,5.0,2324.08,0.0,0.0,355.84,355.84,1601.720350,0.00
...,...,...,...,...,...,...,...,...
7209,11440.0,0.00,0.0,0.0,0.00,0.00,0.000000,153.00
7210,11447.0,0.00,0.0,0.0,0.00,0.00,0.000000,3.01
7211,11456.0,0.00,0.0,0.0,0.00,0.00,0.000000,655.00
7212,11457.0,0.00,0.0,0.0,0.00,0.00,0.000000,2.24


In [843]:
temporaryQuantity['temporary_quantity'] = temporaryQuantity['in_stock'] + temporaryQuantity['ordered_quantity'] + temporaryQuantity[
                        'outsourcing_stock_out'] - temporaryQuantity['need_for_outsourcing'] - temporaryQuantity['need_quantity']

temporaryQuantity               

,product_id,in_stock,ordered_quantity,convert_total_quota,quantity_exported_outsourcing,outsourcing_stock_out,need_for_outsourcing,need_quantity,temporary_quantity
0,1.0,6301.91,0.0,255.2,233.50,0.00,3287.660660,118.41,2895.839340
1,2.0,13742.73,0.0,41.5,6.50,0.00,22719.221511,4384.46,-13360.951511
2,3.0,4437.70,0.0,0.0,0.00,0.00,72.600000,0.00,4365.100000
3,4.0,3270.90,0.0,0.0,0.00,0.00,680.682354,0.00,2590.217646
4,5.0,2324.08,0.0,0.0,355.84,355.84,1601.720350,0.00,1078.199650
...,...,...,...,...,...,...,...,...,...
7209,11440.0,0.00,0.0,0.0,0.00,0.00,0.000000,153.00,-153.000000
7210,11447.0,0.00,0.0,0.0,0.00,0.00,0.000000,3.01,-3.010000
7211,11456.0,0.00,0.0,0.0,0.00,0.00,0.000000,655.00,-655.000000
7212,11457.0,0.00,0.0,0.0,0.00,0.00,0.000000,2.24,-2.240000


## Trừ tạm